<a href="https://colab.research.google.com/github/andreasfloros/ARM-ML-Embedded/blob/main/audio_classifier_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio classifier pipeline

### 1) Download and untar dataset directly to colab:

To change between datasets change the url to one which downloads a .tar file.

**Other important notes:**

**1.** If the audio_data folder already exists in your colab session running the following cell will give an error. That is not a problem, run it anyway.

**2.** To load the audio samples we use the librosa library. Its load function supports a large range of input formats such as WAV, MP3, FLAC, OGG and many others. For the full list of available input formats please refer to the documentation at https://librosa.org/doc/main/generated/librosa.load.html. An important point is that the library can't load README files, which can often be found in datasets in the form of .md and .txt files. Not deleting such files will cause the library to fail when processing the dataset!

In [ ]:
import os
import requests
import tarfile

DATASET_ROOT_DIR = 'audio_data/'
os.mkdir(DATASET_ROOT_DIR)
url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
target_path = 'audio_data/dataset.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

tar = tarfile.open(target_path, "r:gz")
tar.extractall(path='audio_data/')
tar.close()

In [ ]:
DATASET_ROOT_DIR = 'audio_data/'

### 2) Preprocess dataset:

We iterate through the entire dataset, process every audio track using a processing method (either FFT, STFT, or MFCC), and store all of them in a JSON file. Currently, the user is able to control the processing method, expected duration of the audio tracks, the sample rate, as well as other values related to the processing methods.

Potentially, the user might also be able (in the future) to automatically go through all processing methods and use the one which optimizes the current model accuracy.

The process explained above can be completed through the following steps.

**Step 1:** Import necessary libraries

In [ ]:
import os
import librosa
import math
import json
import numpy as np

**Step 1.2:** Download Python Wrapper

In [ ]:
!pip --version

In [ ]:
!pip install --upgrade pip

In [ ]:
!git clone https://github.com/ARM-software/CMSIS_5.git

In [ ]:
# !pip install numpy --user
!ls CMSIS_5/CMSIS/DSP/PythonWrapper
!cd CMSIS_5/CMSIS/DSP/PythonWrapper && pip install . --use-feature=in-tree-build
# python setup.py build_ext --inplace

In [ ]:
import cmsisdsp as dsp

**Step 2:** Function to extend/cut tracks appropriately so that all contain the expected number of samples

In [ ]:
def make_track_correct_size(signal, expected_num_samples_per_track):

    # print('Original track length: {}'.format(len(signal)))
    # if track is shorter than expected, append it with zeros
    if len(signal) < expected_num_samples_per_track:
      num_zeros_to_pad = expected_num_samples_per_track - len(signal)
      zeros = num_zeros_to_pad * [0.]
      extended_signal = np.append(signal, zeros)
      return extended_signal

    # if track is longer than expected, truncate it
    elif len(signal) > expected_num_samples_per_track:
      return signal[:expected_num_samples_per_track]

    # else return the original track 
    else:
      return signal

**Step 3:** Define function to process a single track using the specified method (FFT/STFT/MFCC) and return the data structure containing the result. This function will be called for all tracks within the dataset. Also note that we always return a 1D array by flattening the 2D arrays where necessary. We do this so that we can replicate the exact same preprocessing on Arduino.

In [ ]:
def audio_track_to_features(signal, processing_method, sample_rate, window_size, window_stride, num_mfcc, expected_num_samples_per_track, cmsis_dsp_len):
  # print(signal.shape)
  if processing_method == 'none':
    # if no processing method is selected we only averaged the signal every 32 samples
    averaged = np.mean(signal.reshape(-1, 32), axis=1)
    return averaged

  elif processing_method == 'fft':
    
    # below is 1D fft implementation
    # perform Fast Fourier Transform (FFT)
    fft = np.empty(0)

    for i in range(expected_num_samples_per_track//cmsis_dsp_len):
      # print(i*cmsis_dsp_len)
      # print((i+1)*cmsis_dsp_len-1)
      result = dsp.arm_rfft_fast_f32(rfftf32,signal[i*cmsis_dsp_len:
                                                    (i+1)*cmsis_dsp_len-1],0)
      # print(result.shape)
      # fft = np.fft.fft(signal)

      # calculate abs values on complex numbers to get magnitude
      spectrum = np.abs(result)

      # the spectrum is symmetrical with respect to sample_rate / 2
      # so take half of the spectrum and frequency arrays
      # therefore len(half_spectrum) = sample_rate / 2
      half_spectrum = spectrum[:int(len(spectrum)/2)]
      
      fft = np.append(fft, half_spectrum)
    


    # # average every 16 samples to reduce size of array to 1 / 16 of its original size
    # # e.g. sample_rate = 16k, duration = 1.024s, reduce size from 8192 to 512 
    # averaged = np.mean(fft.reshape(-1, 16), axis=1)

    # # transform to range -128 to 127
    # averaged *= (255.0/averaged.max()) # 0-255
    # averaged -= 128

    # return averaged

    return fft



  elif processing_method == 'stft':
    # perform Short Time Fourier Transform (STFT)
    stft = librosa.stft(signal, 
                        n_fft = window_size, 
                        hop_length = window_stride)

    # calculate abs values on complex numbers to get magnitude
    spectrogram = np.abs(stft)

    # transpose and return the spectrogram matrix
    transposed_spectrogram = spectrogram.transpose()
    return transposed_spectrogram.flatten()


  else: # mfcc
    # perform Mel-Frequency Cepstral Coefficients (MFCC)
    mfcc = librosa.feature.mfcc(signal, 
                                sr = sample_rate, 
                                n_fft = window_size, 
                                n_mfcc = num_mfcc,
                                hop_length = window_stride)
    # transpose and return the mfcc matrix
    transposed_mfcc = mfcc.T
    return transposed_mfcc.flatten()

**Step 4:** Define function to process every audio track and create a JSON file with the entire processed dataset

In [ ]:
def preprocess_entire_dataset(dataset_path, json_path, processing_method, sample_rate, expected_duration, window_size, window_stride, num_mfcc, cmsis_dsp_len):
  # expected duration is in seconds
  expected_num_samples_per_track = int(expected_duration * sample_rate)
  
  # dictionary to later be converted to final json file
  data = {
      'mapping' : [],
      'features' : [],
      'labels' : []
  }

  # we will iterate this for each of the visited sub-directorie in order to
  # give a different label for each of them
  visited_directory_index = 0

  # iterate through all subfolders
  for dirpath, dirnames, filenames in os.walk(dataset_path):

    # # ensure we are not at the dataset root directory
    # # (os.walk provides this directory as well)
    if dirpath is not DATASET_ROOT_DIR:
    # if dirpath == 'audio_data/yes' or dirpath == 'audio_data/no' or dirpath == 'audio_data/bed' or dirpath == 'audio_data/_background_noise_':

      # obtain word labels
      dirpath_components = dirpath.split('/') # audio_data/left => ['audio_data', 'left']
      word_label = dirpath_components[-1]
      data['mapping'].append(word_label)
      print('Processing {}'.format(word_label))

      # access and process files for current word
      for f in filenames:

        # load audio file
        file_path = os.path.join(dirpath, f)
        signal, sample_rate = librosa.load(file_path, sr=sample_rate)

        # extend or cut signal to be equal to the expected size
        signal_correct_size = make_track_correct_size(signal, expected_num_samples_per_track)

        # obtain the features of the audio track using the function defined above
        track_features = audio_track_to_features(signal = signal_correct_size, 
                                                processing_method = processing_method,
                                                sample_rate = sample_rate, 
                                                window_size = window_size, 
                                                window_stride = window_stride, 
                                                num_mfcc = num_mfcc,
                                                expected_num_samples_per_track = expected_num_samples_per_track, 
                                                cmsis_dsp_len = cmsis_dsp_len)

        # append the audio track features to the features field of the dictionary
        data['features'].append(track_features.tolist())

        # append the current directory index as the label of this track
        data['labels'].append(visited_directory_index)
        # print('file_path: {}'.format(file_path))

      # iterate the index before visiting the next directory
      visited_directory_index = visited_directory_index + 1

  print(data['mapping'])
  print(set(data['labels']))
  # create the json file from the dictionary
  with open(json_path, 'w') as fp:
    json.dump(data, fp, indent=4)

**Step 5:** Before running these functions with the code in the following cell we must delete the '.ipynb_checkpoint' files which might otherwise be considered as part of the dataset and interfere with training the model. We also delete the README.md file in the audio_data/background_noise directory.

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
!rm audio_data/_background_noise_/README.md

**Step 6:** Run the preprocessing function above with the desired parameters

In [ ]:
JSON_PATH = 'data.json'
PROCESSING_METHOD = 'fft'
SAMPLE_RATE = 16000
EXPECTED_DURATION = 1.024            # in seconds
WINDOW_SIZE_SAMPLES = 4096            # in samples
WINDOW_STRIDE_SAMPLES = 4096          # in samples
MFCC_COEFF_NUMBER = 13
CMSIS_DSP_LEN = 4096

if (PROCESSING_METHOD == 'fft'):
  nb=4096
  rfftf32=dsp.arm_rfft_fast_instance_f32()
  status=dsp.arm_rfft_fast_init_f32(rfftf32,nb)
  status


preprocess_entire_dataset(dataset_path = DATASET_ROOT_DIR, 
                   json_path = JSON_PATH, 
                   processing_method = PROCESSING_METHOD,
                   sample_rate = SAMPLE_RATE, 
                   expected_duration = EXPECTED_DURATION, 
                   window_size = WINDOW_SIZE_SAMPLES, 
                   window_stride = WINDOW_STRIDE_SAMPLES, 
                   num_mfcc = MFCC_COEFF_NUMBER,
                   cmsis_dsp_len = CMSIS_DSP_LEN)

### 3) Build and evaluate model:

**Step 1:** Import necessary libraries

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Activation, BatchNormalization, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt

**Step 2:** Define functions to load and prepare the dataset for the model

In [ ]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    print('Loading dataset')

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data['features'])
    y = np.array(data['labels'])
    return X, y


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    print('Splitting dataset into training, validation, and test splits')

    # load data
    X, y = load_data(JSON_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets for conv networks to fit conv2D shape specs
    # currently not needed since we reshape everything anyway
    # X_train = X_train[..., np.newaxis]
    # X_validation = X_validation[..., np.newaxis]
    # X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

**Step 3:** Define seperate functions for building a dense and a convolutional model

In [ ]:
def build_conv_model(input_shape, reshape_shape):
    """Generates CNN model
    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = Sequential()

    # reshape input from 1D to 2D
    model.add(Reshape(reshape_shape, input_shape=input_shape))

    # 1st conv layer
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    # 2nd conv layer
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.3))

    # output layer
    model.add(Dense(4, activation='softmax'))

    return model

In [ ]:
def build_dense_model(input_shape):
    """Generates CNN model
    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = Sequential()

    model.add(Dense(16, activation='relu', input_shape=input_shape))
    # model.add(Dropout(0.3))

    model.add(Dense(32, activation='relu'))

    # model.add(Dense(256, activation='relu'))
    # # model.add(Dropout(0.3))

    # model.add(Dense(512, activation='relu'))
    # # model.add(Dropout(0.3))

    # model.add(Dense(256, activation='relu'))

    # model.add(Dense(128, activation='relu'))
    # # model.add(Dropout(0.3))

    model.add(Dense(64, activation='relu'))

    # output layer
    model.add(Dense(36, activation='softmax'))

    return model

**Step 4:** Define function to plot the training history of the model

In [ ]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

**Step 5:** Run all of the above functions:

In [ ]:
# get train, validation, test splits
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)
print('Finished preparing training, validation, and test data')
print('X_train.shape: {}'.format(X_train.shape))

**Step 6:** Since when preprocessing the entire dataset we flattened all 2D arrays to 1D arrays, we calculate the original 2D dimentions in order to Reshape in the first layer of our Neural Network.

In [ ]:
NUM_OF_SAMPLES = SAMPLE_RATE * EXPECTED_DURATION
# none
AVERAGED_SIGNAL_ARRAY_SIZE = int(NUM_OF_SAMPLES / 32) # averaging every 32 samples
# FFT
# FFT_ARRAY_SIZE = int((NUM_OF_SAMPLES) / (2 * 16))  # 2 comes from half spectrum, 16 from averaging every 16 samples
FFT_SLICE_SIZE = CMSIS_DSP_LEN // 2 # each slice has 2048 samples
NUM_OF_SLICES = int(EXPECTED_DURATION * SAMPLE_RATE // CMSIS_DSP_LEN)
# STFT, MFCC
NUM_OF_WINDOW_POSITIONS = math.ceil(NUM_OF_SAMPLES / WINDOW_STRIDE_SAMPLES)
NUM_OF_STFT_FREQUENCIES = int(1 + WINDOW_SIZE_SAMPLES / 2)


if PROCESSING_METHOD == 'none':
  RESHAPE_SHAPE = (1, AVERAGED_SIGNAL_ARRAY_SIZE, 1)
elif PROCESSING_METHOD == 'fft':
  # RESHAPE_SHAPE = (1, FFT_ARRAY_SIZE, 1)
  RESHAPE_SHAPE = (FFT_SLICE_SIZE, NUM_OF_SLICES, 1)
elif PROCESSING_METHOD == 'stft':
  RESHAPE_SHAPE = (NUM_OF_WINDOW_POSITIONS, NUM_OF_STFT_FREQUENCIES, 1)
else: # mfcc
  RESHAPE_SHAPE = (NUM_OF_WINDOW_POSITIONS, MFCC_COEFF_NUMBER, 1)

print('processing method: {}, \nreshape shape: {}'.format(PROCESSING_METHOD, RESHAPE_SHAPE))

In [ ]:
network_is_convolutional = True

# for convolutional model
if network_is_convolutional:
  input_shape = (X_train.shape[1],)
  print("input_shape = {}".format(input_shape))
  model = build_conv_model(input_shape, RESHAPE_SHAPE)
# for dense model
else:
  input_shape = (X_train.shape[1],)
  print("input_shape = {}".format(input_shape))
  model = build_dense_model(input_shape)


# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=20)

# plot accuracy/error for training and validation
plot_history(history)

**Step 7:** Evaluate the model's test accuracy

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

### 4) Save model and convert to desired formats:

**Step 1:** Save keras model as an .h5 file.

In [ ]:
model_name = 'yes_no_bed_background_conv_raw'
model.save(model_name + '.h5')

**Step 2:** For converting to tflite model there is a CHANCE that we require tensorflow 2.2 specifically. So check the version with the cell directly below and if the version isn't 2.2 (or 2.2.something) then run the cell below it. If it is, please don't uninstall and reinstall the same version.

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!pip uninstall tensorflow -y
!pip install  tensorflow==2.2

**Step 3:** Convert the model to a (currently not quantized due to commented out lines) tflite model.

In [ ]:
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)


# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# def representative_dataset_generator():
#   for value in X_test:
#     yield [np.array(value, dtype = np.float32, ndmin=4)]

# converter.representative_dataset = representative_dataset_generator

tflite_model = converter.convert()

# Save the model to disk
open(model_name + '.tflite', "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize(model_name + '.tflite')
print("Model is %d bytes" % basic_model_size)

**Step 4:** Convert model from tflite file to byte array for deployment on Arduino.

In [ ]:
!apt-get -qq install xxd
!xxd -i yesno_conv_fft_normalized.tflite > yesno_conv_fft_normalized.cc

### 5) Extra functions (not required for the pipeline):

**1.** Define function to generate wav from time series to evaluate the Arduino's microphone raw data. To convert to .wav I have currently implemented normalization both to [0, 1] and to [-1, 1]. You can switch between them in the last line by choosing 'normalized' or 'minus_plus_one' as the array to be used for conversion. Both, however, seem to yield the same results.

In [ ]:
import numpy as np
import scipy.io.wavfile as wavf

def time_series_to_wav(time_series):
  # make np array out of list
  time_series_np = np.array(time_series)

  # convert to float32 (this is done because I saw that the dataset samples were float32)
  time_series_np_f = np.float32(time_series_np)
  print('time series max: {}'.format(time_series_np_f.max()))
  print('time series min: {}'.format(time_series_np_f.min()))

  # make everything positive to normalize to [0, 1] in next step 
  # perhaps this doesn't make sense and [-1, 1] is more correct?
  allpositives = time_series_np_f + abs(time_series_np_f.min())
  print('all positives max: {}'.format(allpositives.max()))
  print('all positives min: {}'.format(allpositives.min()))

  # normalize to [0, 1])
  normalized = allpositives / abs(allpositives.max())
  print('normalized max: {}'.format(normalized.max()))
  print('normalized min: {}'.format(normalized.min()))

  # normalize to [-1, 1]
  minus_plus_one = 2 * ( (time_series_np_f - time_series_np_f.min()) / (time_series_np_f.max() - time_series_np_f.min()) ) - 1
  print('minus plus one max: {}'.format(minus_plus_one.max()))
  print('minus plus one min: {}'.format(minus_plus_one.min()))


  sampling_frequency = 16000
  output_wav_file = 'minus_plus_one.wav'

  wavf.write(output_wav_file, sampling_frequency, minus_plus_one)

In [ ]:
time_series = [271, 270, 270, 269, 270, 270, 269, 269, 269, 269, 269, 268, 269, 268, 268, 268, 268, 267, 268, 267, 267, 267, 266, 267, 267, 268, 268, 267, 267, 267, 267, 266, 267, 267, 268, 268, 268, 267, 267, 267, 267, 267, 269, 267, 267, 266, 268, 268, 268, 268, 267, 268, 267, 268, 268, 269, 269, 269, 268, 270, 269, 1306, 1303, 1299, 1295, 1289, 1281, 1273, 1264, 1254, 1244, 1231, 1218, 1205, 1191, 1179, 1165, 1153, 1143, 1133, 1126, 1121, 1116, 1111, 1109, 1105, 1103, 1098, 1096, 1094, 1091, 1093, 1095, 1099, 1104, 1111, 1120, 1129, 1138, 1144, 1151, 1153, 1155, 1154, 1153, 1150, 1149, 1147, 1149, 1155, 1163, 1179, 1201, 1229, 1262, 1299, 1337, 1376, 1415, 1453, 1493, 1532, 1571, 1609, 1645, 1677, 1704, 1723, 1737, 1745, 1748, 1749, 1749, 1751, 1755, 1765, 1778, 1795, 1817, 1842, 1868, 1892, 1914, 1931, 1941, 1942, 1934, 1918, 1895, 1866, 1834, 1798, 1763, 1727, 1689, 1654, 1617, 1581, 1547, 1513, 1479, 1446, 1415, 1384, 1354, 1325, 1298, 1270, 1247, 1223, 1200, 1178, 1158, 1139, 1121, 1103, 1087, 1073, 1058, 1044, 1033, 1022, 1012, 1002, 995, 986, 980, 975, 970, 966, 962, 960, 957, 955, 953, 954, 953, 951, 951, 950, 950, 951, 950, 951, 952, 952, 953, 954, 955, 955, 956, 958, 958, 959, 961, 962, 963, 964, 967, 966, 968, 969, 971, 972, 973, 973, 973, 974, 972, 972, 971, 970, 969, 969, 966, 964, 960, 957, 954, 950, 946, 942, 937, 930, 925, 918, 913, 905, 899, 891, 883, 877, 869, 863, 855, 848, 843, 838, 832, 829, 825, 821, 819, 817, 817, 815, 815, 815, 815, 814, 816, 814, 815, 816, 817, 817, 817, 817, 816, 816, 815, 814, 815, 813, 813, 811, 809, 809, 807, 805, 803, 800, 799, 798, 795, 794, 793, 792, 790, 788, 787, 786, 785, 784, 782, 781, 781, 780, 780, 779, 780, 780, 780, 779, 779, 779, 776, 775, 773, 771, 769, 764, 762, 698, 698, 698, 697, 699, 699, 700, 700, 700, 700, 700, 700, 700, 701, 702, 703, 702, 703, 703, 704, 704, 704, 704, 704, 705, 706, 705, 705, 706, 705, 705, 705, 707, 706, 706, 707, 707, 708, 708, 707, 707, 707, 708, 708, 709, 709, 709, 709, 709, 709, 709, 709, 709, 709, 708, 709, 709, 708, 709, 708, 709, 709, 709, 710, 709, 708, 709, 709, 709, 708, 709, 709, 709, 709, 708, 709, 707, 708, 708, 708, 707, 708, 709, 708, 707, 707, 707, 708, 707, 707, 707, 706, 706, 706, 706, 705, 705, 705, 705, 704, 705, 704, 704, 704, 704, 702, 702, 701, 703, 701, 702, 701, 702, 702, 701, 702, 700, 701, 700, 701, 700, 700, 700, 700, 700, 701, 700, 700, 699, 699, 699, 700, 698, 699, 698, 698, 698, 697, 697, 696, 696, 696, 695, 696, 695, 695, 695, 694, 693, 694, 694, 694, 693, 692, 693, 692, 692, 692, 692, 691, 691, 690, 689, 690, 690, 689, 689, 688, 688, 688, 687, 688, 687, 687, 687, 687, 687, 686, 686, 686, 685, 685, 686, 686, 685, 685, 685, 685, 684, 684, 684, 682, 683, 683, 682, 683, 682, 683, 683, 682, 682, 682, 682, 681, 680, 681, 681, 679, 679, 680, 679, 679, 678, 679, 679, 678, 678, 678, 677, 678, 678, 678, 679, 678, 678, 677, 677, 678, 677, 678, 677, 676, 677, 676, 676, 675, 676, 675, 675, 674, 674, 674, 674, 673, 673, 674, 674, 674, 672, 673, 672, 673, 672, 672, 673, 672, 671, 671, 670, 670, 670, 671, 669, 669, 668, 669, 669, 669, 668, 667, 668, 668, 667, 667, 667, 666, 666, 666, 667, 667, 666, 667, 667, 666, 666, 666, 665, 666, 665, 665, 665, 665, 666, 664, 664, 665, 664, 665, 665, 664, 665, 665, 664, 665, 664, 664, 665, 665, 665, 665, 665, 664, 666, 665, 666, 666, 665, 666, 666, 666, 666, 667, 667, 669, 668, 670, 211, 210, 211, 210, 210, 208, 209, 208, 207, 208, 207, 206, 206, 206, 205, 205, 204, 204, 203, 203, 202, 202, 202, 202, 202, 201, 201, 200, 199, 199, 198, 198, 197, 197, 197, 196, 195, 196, 195, 195, 193, 193, 193, 193, 194, 192, 191, 191, 192, 191, 189, 190, 189, 189, 190, 188, 187, 188, 187, 187, 185, 186, 185, 186, 184, 184, 184, 183, 183, 182, 181, 181, 180, 181, 180, 179, 179, 178, 178, 177, 177, 176, 176, 175, 176, 175, 175, 175, 173, 173, 173, 171, 172, 171, 172, 171, 170, 171, 170, 170, 168, 169, 168, 167, 167, 167, 167, 167, 166, 166, 167, 166, 167, 165, 164, 164, 164, 164, 164, 163, 162, 161, 160, 159, 160, 159, 159, 159, 158, 158, 157, 156, 155, 155, 154, 155, 154, 153, 153, 153, 152, 153, 152, 151, 151, 151, 151, 150, 150, 149, 150, 149, 150, 149, 148, 148, 148, 148, 147, 146, 146, 147, 145, 145, 146, 144, 144, 143, 144, 144, 142, 143, 143, 142, 142, 141, 140, 140, 140, 140, 140, 139, 138, 138, 138, 137, 137, 137, 136, 136, 135, 136, 135, 134, 135, 134, 134, 134, 133, 133, 133, 133, 132, 132, 132, 131, 130, 130, 131, 129, 129, 129, 129, 128, 128, 128, 127, 127, 127, 126, 125, 126, 126, 125, 126, 125, 125, 125, 124, 122, 123, 123, 122, 122, 121, 121, 121, 121, 120, 120, 119, 118, 119, 119, 118, 118, 117, 116, 116, 115, 114, 115, 114, 112, 112, 112, 111, 111, 110, 110, 109, 108, 109, 109, 108, 108, 108, 107, 108, 107, 106, 106, 105, 106, 105, 105, 105, 105, 103, 103, 103, 102, 102, 101, 100, 100, 100, 99, 99, 98, 97, 96, 97, 97, 97, 97, 96, 96, 95, 95, 95, 95, 93, 94, 92, 93, 92, 92, 91, 91, 91, 90, 90, 89, 89, 88, 88, 87, 86, 86, 86, 85, 86, 85, 84, 85, -304, -304, -305, -305, -306, -307, -307, -307, -307, -307, -309, -308, -309, -310, -309, -310, -310, -311, -311, -312, -312, -313, -313, -313, -313, -315, -314, -314, -315, -317, -316, -316, -317, -317, -318, -317, -318, -319, -319, -320, -320, -320, -321, -321, -321, -323, -323, -323, -322, -639, -639, -640, -639, -639, -639, -638, -638, -639, -639, -639, -638, -639, -639, -638, -638, -638, -639, -640, -639, -639, -639, -639, -640, -639, -640, -640, -640, -639, -639, -639, -638, -639, -639, -639, -638, -640, -640, -638, -639, -638, -639, -639, -640, -640, -639, -640, -640, -639, -640, -640, -640, -640, -641, -640, -639, -638, -640, -639, -639, -640, -342, -343, -342, -343, -343, -343, -342, -343, -344, -345, -345, -346, -346, -346, -346, -347, -347, -348, -348, -350, -350, -349, -350, -350, -349, -350, -351, -350, -352, -353, -353, -352, -353, -354, -355, -355, -357, -356, -357, -357, -357, -357, -358, -358, -359, -358, -359, -360, -360, -361, -361, -361, -361, -361, -362, -362, -362, -363, -362, -363, -363, -364, -365, -365, -367, -367, -367, -367, -368, -368, -368, -369, -370, -371, -372, -372, -371, -373, -372, -373, -373, -374, -375, -374, -375, -376, -377, -378, -377, -377, -379, -378, -379, -379, -380, -380, -381, -381, -382, -382, -382, -383, -382, -384, -383, -385, -385, -385, -386, -385, -665, -666, -665, -665, -665, -664, -664, -665, -665, -666, -665, -665, -665, -665, -666, -666, -666, -666, -665, -665, -665, -665, -664, -665, -665, -664, -665, -665, -665, -665, -666, -665, -665, -665, -665, -664, -664, -665, -665, -665, -666, -665, -665, -666, -667, -665, -666, -666, -666, -666, -666, -666, -666, -667, -667, -668, -668, -668, -669, -669, -668, -668, -669, -669, -667, -667, -669, -669, -668, -668, -669, -669, -667, -668, -668, -667, -668, -668, -668, -667, -668, -668, -668, -669, -670, -670, -669, -668, -670, -669, -670, -670, -670, -670, -671, -670, -670, -670, -669, -670, -671, -671, -670, -671, -672, -671, -673, -673, -673, -673, -673, -674, -672, -673, -673, -673, -674, -673, -675, -674, -673, -674, -674, -674, -674, -674, -675, -675, -675, -675, -674, -675, -675, -675, -674, -675, -674, -675, -676, -676, -675, -676, -675, -674, -675, -675, -676, -675, -675, -677, -676, -676, -675, -675, -676, -675, -674, -675, -675, -675, -674, -675, -675, -674, -675, -675, -676, -676, -675, -675, -675, -675, -675, -675, -675, -675, -676, -675, -676, -676, -676, -676, -675, -675, -676, -676, -676, -676, -675, -675, -675, -675, -676, -675, -675, -676, -676, -676, -676, -676, -675, -674, -675, -674, -675, -674, -674, -673, -674, -674, -673, -674, -673, -673, -672, -671, -672, -671, -671, -594, -594, -594, -594, -595, -595, -594, -595, -594, -594, -595, -594, -593, -593, -594, -594, -593, -595, -594, -594, -594, -593, -594, -593, -594, -593, -593, -593, -592, -594, -593, -593, -595, -593, -593, -593, -593, -593, -591, -593, -592, -592, -592, -591, -591, -592, -592, -591, -592, -592, -592, -592, -592, -591, -590, -590, -591, -591, -590, -590, -590, -590, -590, -590, -590, -591, -590, -590, -590, -590, -590, -590, -590, -590, -590, -589, -591, -590, -590, -590, -590, -590, -590, -591, -591, -590, -591, -590, -589, -589, -589, -590, -589, -589, -589, -589, -589, -589, -589, -589, -589, -588, -589, -588, -588, -587, -588, -587, -588, -587, -587, -587, -587, -586, -586, -586, -587, -586, -586, -585, -585, -586, -584, -585, -585, -585, -584, -584, -584, -584, -583, -584, -584, -583, -584, -584, -583, -582, -582, -583, -583, -583, -583, -583, -582, -583, -583, -583, -583, -582, -582, -582, -581, -581, -581, -581, -580, -579, -579, -580, -581, -580, -580, -580, -580, -580, -580, -579, -580, -579, -579, -579, -579, -579, -579, -579, -578, -579, -578, -578, -577, -578, -579, -578, -578, -578, -576, -576, -575, -575, -575, -574, -574, -574, -574, -574, -575, -574, -573, -574, -572, -572, -572, -572, -571, -571, -570, -571, -571, -570, -570, -571, -571, -571, -570, -570, -570, -571, -570, -570, -570, -445, -444, -444, -444, -444, -445, -444, -444, -444, -445, -445, -445, -444, -444, -444, -444, -444, -445, -444, -444, -445, -444, -445, -446, -445, -445, -446, -445, -445, -444, -445, -444, -444, -445, -444, -445, -445, -445, -445, -445, -446, -445, -445, -446, -446, -446, -445, -446, -445, -445, -445, -446, -445, -445, -444, -445, -445, -445, -445, -445, -444, -444, -444, -444, -444, -445, -445, -444, -443, -444, -444, -443, -444, -444, -444, -445, -445, -444, -444, -444, -444, -444, -443, -444, -444, -445, -444, -444, -445, -445, -444, -445, -445, -445, -445, -445, -444, -445, -444, -444, -443, -444, -444, -443, -444, -443, -443, -443, -443, -442, -443, -443, -443, -443, -444, -443, -443, -443, -443, -442, -443, -442, -442, -442, -443, -442, -442, -443, -443, -443, -441, -442, -442, -442, -441, -442, -441, -442, -442, -442, -442, -441, -443, -443, -442, -443, -443, -443, -443, -443, -444, -443, -444, -443, -443, -443, -443, -444, -443, -444, -444, -445, -444, -445, -445, -444, -445, -444, -445, -445, -444, -444, -445, -445, -444, -445, -445, -445, -445, -445, -444, -444, -444, -445, -445, -445, -445, -445, -444, -445, -445, -444, -445, -444, -445, -445, -445, -445, -445, -445, -445, -445, -445, -445, -444, -444, -444, -444, -445, -444, -445, -444, -444, -444, -445, -445, -445, -445, -444, -444, -444, -444, -443, -389, -390, -390, -390, -389, -389, -388, -388, -388, -388, -388, -388, -388, -388, -387, -387, -387, -387, -388, -387, -388, -387, -387, -386, -386, -388, -386, -387, -386, -386, -387, -387, -387, -387, -386, -386, -386, -387, -386, -385, -386, -386, -386, -386, -386, -386, -387, -387, -386, -386, -386, -385, -386, -386, -386, -386, -385, -386, -386, -385, -385, -385, -384, -385, -384, -384, -384, -385, -383, -384, -384, -383, -383, -383, -382, -383, -383, -383, -384, -383, -384, -383, -383, -383, -383, -383, -383, -383, -383, -383, -383, -384, -383, -384, -383, -382, -384, -384, -384, -383, -384, -383, -384, -384, -383, -383, -384, -384, -384, -385, -385, -384, -385, -384, -385, -384, -384, -384, -384, -385, -385, -384, -384, -385, -384, -384, -384, -384, -385, -384, -384, -383, -384, -384, -383, -382, -384, -383, -384, -384, -383, -383, -383, -383, -383, -382, -383, -383, -382, -382, -382, -381, -382, -383, -383, -383, -383, -383, -383, -383, -383, -383, -382, -382, -382, -382, -382, -382, -382, -383, -383, -383, -383, -383, -383, -383, -383, -382, -383, -383, -383, -384, -384, -384, -384, -383, -383, -382, -383, -383, -249, -250, -249, -248, -246, -246, -245, -244, -245, -244, -244, -241, -239, -238, -238, -236, -235, -234, -233, -231, -229, -227, -225, -222, -219, -218, -216, -216, -213, -212, 195, 79, -60, -116, -55, 68, 155, 144, 59, -32, -73, -60, -41, -57, -110, -155, -146, -80, -5, 13, -57, -175, -262, -260, -185, -103, -75, -113, -168, -196, -184, -164, -295, -295, -294, -294, -294, -294, -294, -294, -294, -293, -294, -294, -293, -293, -293, -294, -294, -294, -292, -294, -293, -293, -292, -292, -292, -293, -291, -291, -291, -291, -291, -291, -291, -290, -290, -291, -290, -291, -292, -291, -291, -290, -290, -291, -292, -291, -290, -290, -290, -290, -290, -290, -290, -288, -289, -289, -290, -290, -290, -289, -289, -288, -289, -290, -290, -290, -290, -289, -289, -289, -288, -288, -287, -288, -287, -289, -288, -289, -287, -288, -289, -287, -287, -287, -286, -287, -287, -287, -286, -286, -286, -286, -286, -286, -287, -287, -287, -287, -288, -288, -287, -287, -287, -288, -287, -288, -288, -288, -288, -289, -289, -288, -289, -290, -289, -290, -290, -290, -290, -289, -289, -289, -290, -290, -290, -290, -290, -290, -289, -289, -289, -290, -290, -290, -290, -289, -289, -289, -288, -290, -290, -290, -290, -290, -291, -290, -291, -291, -290, -289, -290, -290, -291, -290, -289, -289, -288, -290, -291, -290, -290, -289, -288, -288, -288, -288, -289, -289, -288, -288, -288, -288, -286, -287, -287, -287, -288, -287, -289, -288, -288, -288, -288, -288, -288, -289, -289, -290, -291, -291, -290, -291, -290, -549, -534, -495, -435, -383, -374, -413, -471, -496, -449, -341, -226, -170, -206, -297, -360, -343, -254, -150, -104, -133, -192, -218, -175, -98, -59, -89, -162, -210, -184, -100, -21, -3, -56, -130, -163, -141, -98, -79, -96, -119, -120, -91, -62, -71, -121, -179, -198, -168, -109, -69, -75, -120, -169, -195, -189, -168, -156, -162, -178, -184, -175, -157, -150, -166, -198, -226, -228, -202, -170, -154, -167, -195, -216, -224, -216, -207, -207, -212, -214, -206, -197, -198, -212, -228, -233, -224, -204, -187, -176, -177, -176, -168, -157, -149, -143, -134, -116, -91, -72, -64, -67, -73, -71, -54, -30, -13, -11, -12, -14, -7, -3, -4, -14, -26, -32, -26, -15, -13, -25, -47, -70, -81, -85, -84, -89, -101, -117, -132, -146, -160, -180, -201, -218, -232, -241, -250, -265, -289, -316, -342, -365, -386, -407, -430, -451, -471, -494, -517, -539, -564, -587, -605, -614, -615, -611, -602, -578, -531, -462, -410, -400, -437, -494, -513, -453, -329, -208, -149, -188, -280, -340, -321, -223, -112, -64, -95, -156, -180, -130, -47, -5, -43, -126, -178, -144, -44, 45, 60, -6, -88, -120, -91, -41, -22, -45, -73, -67, -25, 14, 4, -58, -127, -152, -115, -49, -6, -15, -64, -117, -144, -139, -125, -122, -137, -155, 743, 1428, 1374, 579, -232, -396, 43, 626, 909, 756, 312, -311, -924, -1260, -1091, -477, 117, 236, -99, -444, -384, 21, 330, 252, -22, -62, 381, 1017, 1305, 1034, 324, -219, -283, 8, 366, 418, 197, -197, -480, -527, -423, -360, -438, -548, -450, -127, 387, 667, 626, 275, 33, 202, 699, 1148, 1247, 1007, 760, 712, 836, 933, 779, 507, 351, 440, 680, 777, 619, 355, 171, 206, 410, 625, 745, 732, 695, 702, 750, 777, 766, 696, 637, 574, 571, 561, 499, 326, 119, -38, -114, -122, -117, -173, -268, -366, -397, -371, -356, -389, -405, -410, -346, -322, -324, -404, -518, -589, -662, -734, -858, -1005, -1096, -1224, -1313, -1430, -1521, -1585, -1660, -1711, -1690, -1756, -1712, -1576, -829, 599, 2185, 2949, 1944, -136, -1562, -1010, 1155, 2943, 2680, 839, -772, -733, 403, 797, -312, -1966, -2271, -686, 1344, 1979, 796, -904, -1468, -580, 778, 1462, 1229, 814, 840, 1251, 1485, 1212, 622, 108, -113, -64, 132, 364, 476, 305, -203, -885, -1324, -1207, -608, 54, 326, 163, -86, -48, 342, 777, 853, 542, 228, 349, 937, 1472, 1410, 706, -119, -485, -245, 153, 310, 95, -256, -402, -402, -316, -262, -266, -216, -92, 193, 679, 1047, 1182, 893, 495, 401, 709, 1182, 1409, 1181, 814, 624, 696, 823, 705, 427, 226, 276, 555, 787, 821, 680, 519, 484, 601, 790, 977, 1079, 1077, 1007, 936, 925, 915, 826, 629, 438, 366, 423, 480, 393, 149, -105, -207, -179, -62, -105, -156, -230, -197, -114, -84, -140, -235, -295, -286, -223, -203, -249, -367, -486, -609, -748, -879, -1074, -1213, -1303, -1333, -1320, -1362, 929, 827, 898, 924, 888, 727, 520, 415, 381, 381, 455, 551, 670, 829, 914, 1032, 1193, 1291, 1320, 1240, 1135, 1138, 1181, 1171, 1099, 972, 882, 868, 834, 759, 643, 510, 425, 371, 347, 373, 415, 449, 466, 464, 481, 514, 517, 464, 372, 289, 261, 257, 224, 171, 129, 112, 102, 46, -48, -157, -251, -335, -391, -423, -425, -466, -539, -635, -697, -723, -749, -811, -893, -987, -1053, -1102, -1156, -1203, -1227, -1246, -1209, -1120, -626, 474, 1880, 2932, 2906, 1966, 1029, 701, 916, 932, 306, -564, -860, -368, 451, 799, 464, -28, -239, -148, -39, -178, -328, -180, 354, 1082, 1656, 1848, 1770, 1586, 1388, 1191, 872, 447, 82, -98, -34, 117, 160, 103, 66, 112, 134, -65, -453, -706, -568, -101, 355, 521, 525, 670, 1053, 1441, 1496, 1164, 722, 463, 429, 446, 349, 216, 164, 203, 231, 122, -36, -136, -166, -194, -223, -198, 29, 376, 672, 829, 878, 925, 1110, 1243, 1193, 1007, 741, 633, 651, 618, 602, 551, 519, 547, 569, 587, 662, 679, 689, 723, 758, 899, 1096, 1195, 1249, 1221, 1213, 1268, 1260, 1145, 995, 832, 735, 697, 649, 613, 580, 525, 498, 465, 452, 463, 451, 404, 357, 331, 374, 448, 475, 463, 432, 403, 407, 355, 272, 170, 82, 42, 3, -47, -86, -132, -160, -194, -257, -314, -358, -439, -513, -578, -616, -602, -624, -677, -729, -795, -866, -942, -1061, -1193, -1313, -1395, -1413, -1361, -1203, -644, 437, 1739, 2717, 2767, 2077, 1381, 1102, 1159, 963, 215, -627, -904, -489, 188, 495, 314, 84, 63, 173, 157, -136, -406, -355, 59, 653, 1147, 1407, 1554, 1667, 1722, 1638, 1306, 794, 301, -37, -167, -202, -247, -259, -142, 96, 272, 183, -121, -362, 1233, 1242, 1250, 1256, 1260, 1260, 1259, 1253, 1244, 1233, 1217, 1199, 1179, 1161, 1143, 1124, 1106, 1092, 1080, 1069, 1060, 1056, 1056, 1057, 1061, 1066, 1070, 1073, 1074, 1074, 1072, 1065, 1057, 1050, 1040, 1030, 1023, 1020, 1018, 1018, 1025, 1037, 1051, 1067, 1083, 1104, 1125, 1146, 1170, 1188, 1203, 1215, 1224, 1233, 1241, 1245, 1249, 1250, 1251, 1251, 1249, 1244, -588, -607, -665, -734, -818, -907, -990, -1079, -1152, -1201, -1234, -1256, -1278, -1308, -1321, -1339, -1369, -1405, -1449, -1497, -1552, -1592, -1612, -1609, -1595, -1589, -1568, -1552, -1536, -1524, -1533, -1573, -1643, -1724, -1800, -1866, -1951, -2031, -2112, -2162, -2095, -1765, -1208, -633, -286, -169, -42, 242, 526, 562, 230, -262, -666, -924, -1188, -1556, -1934, -2103, -2004, -1816, -1696, -1612, -1424, -1100, -770, -579, -548, -591, -633, -648, -671, -740, -842, -919, -920, -844, -749, -686, -649, -623, -595, -598, -697, -918, -1182, -1383, -1497, -1607, -1776, -1932, -1935, -1750, -1485, -1253, -1068, -856, -591, -341, -197, -184, -238, -308, -387, -553, -788, -1017, -1165, -1214, -1251, -1315, -1339, -1277, -1133, -1006, -968, -981, -984, -967, -962, -995, -1045, -1062, -1067, -1009, -912, -779, -600, -451, -320, -239, -206, -156, -167, -240, -406, -588, -749, -872, -995, -1086, -1120, -1117, -1062, -983, -856, -727, -609, -520, -472, -450, -456, -472, -518, -595, -671, -718, -719, -706, -698, -681, -658, -634, -623, -632, -658, -716, -794, -897, -995, -1082, -1139, -1171, -1178, -1162, -1115, -1040, -939, -837, -751, -684, -642, -629, 771, 768, 763, 757, 750, 741, 733, 718, 704, 690, 676, 664, 652, 642, 637, 635, 633, 632, 633, 637, 644, 648, 652, 653, 654, 654, 652, 647, 642, 636, 632, 626, 622, 620, 618, 620, 621, 629, 638, 642, 653, 662, 670, 674, 676, 680, 682, 681, 683, 687, 687, 687, 686, 688, 687, 689, 695, 699, 700, 700, 700, 706, 709, 709, 714, 715, 717, 717, 719, 725, 728, 731, 736, 742, 743, 746, 748, 754, 757, 759, 759, 759, 758, 758, 759, 761, 761, 762, 765, 769, 774, 777, 782, 788, 791, 793, 797, 798, 798, 795, 792, 793, 792, 789, 789, 788, 789, 790, 793, 795, 800, 805, 810, 815, 819, 821, 824, 827, 827, 826, 824, 824, 824, 821, 822, 820, 819, 818, 819, 819, 819, 818, 818, 819, 819, 819, 818, 816, 815, 813, 812, 808, 805, 802, 800, 796, 794, 793, 792, 789, 785, 780, 777, 773, 770, 767, 762, 758, 751, 747, 741, 738, 736, 734, 732, 729, 724, 722, 720, 720, 719, 718, 717, 718, 720, 723, 724, 727, 727, 725, 725, 728, 735, 742, 747, 752, 753, 757, 762, 765, 768, 770, 770, 769, 765, 762, 762, 762, 760, 756, 751, 747, 747, 746, 749, 750, 754, 759, 766, 774, 783, 793, 804, 812, 818, 826, 835, 844, 843, 844, 844, 845, 845, 844, 847, 849, 849, 851, 855, 861, 868, 877, 885, 891, 898, 908, 917, 923, 928, 933, 941, 944, 947, 948, 950, 950, 952, 955, 958, 961, 963, 967, 970, 976, 980, 987, 991, 994, 998, 1002, 1007, 1009, 1011, 1012, 1013, 1014, 1014, 1014, 1015, 1015, 1016, 1015, 1017, 1019, 1021, 1021, 1022, 1023, 1024, 1026, 1026, 1025, 1023, 1022, 1018, 1016, 1013, 1010, 1006, 1002, 997, 995, 991, 988, 983, 980, 978, 975, 973, 971, 969, 968, 967, 967, 966, 965, 966, 964, 962, 960, 959, 959, 957, 955, 951, 948, 945, 944, 944, 943, 941, 937, 933, 929, 925, 922, 919, 1120, 1122, 1126, 1131, 1136, 1139, 1143, 1148, 1153, 1160, 1166, 1170, 1171, 1172, 1172, 1173, 1173, 1172, 1170, 1170, 1171, 1171, 1171, 1172, 1172, 1173, 1178, 1182, 1185, 1187, 1189, 1191, 1189, 1190, 1188, 1185, 1181, 1179, 1177, 1174, 1172, 1169, 1166, 1163, 1162, 1161, 1159, 1155, 1152, 1151, 1149, 1146, 1144, 1142, 1140, 1137, 1134, 1130, 1125, 1122, 1120, 1117, 1115, 1113, 1111, 1110, 1106, 1101, 1093, 1083, 1071, 1059, 1046, 1030, 1014, 999, 983, 966, 953, 941, 930, 922, 916, 912, 910, 910, 912, 915, 920, 924, 930, 937, 941, 947, 953, 960, 963, 968, 972, 975, 977, 980, 984, 986, 989, 991, 992, 992, 993, 994, 993, 993, 994, 993, 994, 993, 992, 992, 990, 988, 985, 983, 979, 976, 970, 966, 962, 956, 951, 944, 939, 932, 925, 916, 908, 900, 894, 887, 881, 876, 872, 867, 863, 860, 857, 854, 849, 846, 843, 842, 840, 841, 839, 838, 838, 838, 840, 839, 840, 840, 839, 841, 841, 842, 844, 844, 846, 847, 851, 853, 855, 858, 861, 865, 868, 872, 875, 877, 879, 881, 885, 888, 893, 897, 901, 904, 907, 910, 910, 910, 908, 906, 902, 901, 899, 897, 895, 894, 890, 889, 887, 884, 882, 879, 875, 872, 869, 865, 861, 855, 851, 848, 843, 838, 836, 833, 834, 834, 834, 838, 841, 846, 851, 856, 861, 865, 870, 875, 879, 883, 885, 887, 888, 887, 886, 885, 883, 883, 881, 880, 880, 880, 881, 880, 880, 879, 878, 877, 876, 872, 870, 867, 863, 859, 854, 849, 843, 839, 835, 831, 829, 826, 825, 825, 826, 827, 828, 830, 831, 833, 836, 839, 843, 848, 852, 856, 859, 864, 866, 869, 873, 878, 880, 883, 886, 889, 892, 894, 895, 897, 898, 899, 898, 900, 901, 902, 904, 905, 907, 907, 910, 910, 912, 914, 916, 917, 920, 922, 924, 924, 925, 926, 925, 926, 927, 927, 926, 926, 926, 927, 929, 929, 929, 930, 931, 931, 932, 934, 935, 1030, 1030, 1031, 1031, 1033, 1033, 1032, 1032, 1033, 1034, 1033, 1035, 1033, 1034, 1035, 1036, 1035, 1036, 1036, 1036, 1036, 1036, 1037, 1038, 1037, 1037, 1038, 1038, 1039, 1038, 1040, 1039, 1040, 1039, 1040, 1040, 1041, 1043, 1042, 1042, 1043, 1043, 1043, 1045, 1045, 1045, 1045, 1047, 1047, 1047, 1049, 1048, 1049, 1048, 1050, 1050, 1050, 1052, 1052, 1052, 1051, 1053, 1052, 1052, 1052, 1052, 1052, 1052, 1052, 1053, 1051, 1051, 1051, 1051, 1051, 1051, 1050, 1049, 1049, 1048, 1048, 1047, 1047, 1046, 1045, 1044, 1044, 1042, 1042, 1041, 1040, 1040, 1038, 1037, 1037, 1036, 1035, 1035, 1034, 1033, 1033, 1031, 1030, 1030, 1029, 1029, 1027, 1027, 1026, 1026, 1024, 1023, 1022, 1021, 1021, 1019, 1017, 1017, 1017, 1016, 1015, 1014, 1012, 1012, 135, 134, 134, 133, 134, 133, 133, 133, 133, 133, 132, 132, 131, 132, 131, 132, 131, 131, 131, 130, 131, 130, 130, 129, 130, 130, 130, 129, 130, 129, 129, 129, 129, 128, 128, 128, 128, 128, 127, 126, 126, 127, 125, 126, 125, 125, 124, 124, 124, 124, 124, 123, 124, 123, 123, 122, 122, 121, 122, 122, 122, 122, 123, 939, 939, 937, 936, 935, 933, 932, 930, 929, 928, 928, 926, 924, 923, 921, 919, 918, 917, 916, 913, 913, 912, 910, 910, 908, 906, 904, 904, 903, 902, 900, 898, 898, 896, 894, 893, 892, 891, 888, 888, 887, 885, 884, 882, 881, 880, 879, 877, 876, 874, 874, 872, 872, 870, 869, 868, 866, 865, 863, 862, 860, 858, 857, 856, 855, 854, 853, 852, 850, 849, 848, 847, 845, 844, 843, 842, 840, 839, 838, 836, 835, 833, 832, 830, 829, 827, 826, 825, 823, 822, 820, 819, 817, 816, 814, 812, 811, 810, 808, 807, 805, 803, 801, 801, 798, 796, 796, 794, 793, 790, 789, 788, 785, 784, 781, 781, 779, 779, 776, 775, 774, 773, 771, 769, 769, 767, 767, 766, 764, 762, 761, 760, 758, 758, 757, 756, 755, 754, 753, 753, 751, 750, 446, 445, 445, 445, 445, 445, 444, 444, 443, 442, 442, 441, 441, 440, 440, 439, 438, 438, 438, 438, 437, 437, 436, 435, 435, 434, 433, 433, 432, 432, 432, 431, 431, 431, 430, 428, 429, 429, 428, 428, 428, 428, 426, 427, 426, 425, 425, 423, 424, 423, 423, 422, 423, 422, 421, 420, 419, 420, 419, 419, 419, 417, 416, 416, 416, 416, 414, 414, 414, 413, 413, 411, 412, 411, 410, 410, 410, 408, 408, 407, 408, 407, 407, 406, 406, 406, 406, 405, 404, 404, 403, 402, 402, 401, 401, 400, 400, 399, 398, 400, 399, 398, 398, 397, 397, 396, 396, 394, 394, 395, 395, 393, 391, 392, 391, 392, 391, 390, 390, 390, 389, 388, 388, 387, 387, 387, 386, 386, 385, 385, 385, 384, 384, 383, 382, 382, 382, 382, 380, 380, 380, 378, 379, 378, 378, 378, 377, 377, 377, 377, 376, 376, 374, 374, 375, 373, 374, 374, 373, 373, 372, 372, 372, 371, 371, 370, 370, 370, 370, 369, 370, 369, 369, 368, 368, 369, 367, 368, 368, 367, 366, 366, 366, 366, 366, 365, 365, 364, 364, 363, 363, 362, 362, 361, 360, 359, 358, 358, 358, 358, 358, 358, 358, 356, 357, 356, 355, 354, 354, 354, 354, 352, 353, 352, 352, 352, 350, 349, 349, 349, 349, 348, 348, 347, 346, 346, 346, 346, 344, 344, 344, 344, 343, 342, 342, 342, 342, 341, 342, 341, 341, 341, 340, 340, 338, 339, 339, 337, 337, 337, 336, 336, 336, 335, 334, 334, 333, 333, 332, 332, 331, 330, 331, 329, 330, 329, 328, 328, 329, 327, 326, 326, 326, 326, 325, 324, 325, 324, 325, 324, 323, 322, 321, 321, 321, 321, 320, 319, 318, 318, 317, 317, 317, 316, 316, 316, 315, 314, 315, 314, 314, 313, 313, 311, 311, 311, 309, 310, 309, 308, 308, 309, 308, 307, 306, 306, 305, 305, 305, 305, 305, 303, 302, 303, 302, 301, 300, 300, 299, 299, -21, -22, -23, -24, -24, -23, -24, -24, -25, -25, -25, -26, -27, -26, -27, -27, -28, -27, -28, -29, -29, -30, -30, -30, -30, -31, -31, -32, -31, -32, -33, -34, -33, -33, -35, -35, -35, -36, -36, -37, -38, -37, -38, -38, -38, -38, -38, -39, -40, -39, -40, -40, -40, -41, -42, -41, -42, -42, -41, -42, -42, -43, -43, -43, -45, -45, -45, -46, -46, -46, -47, -47, -48, -49, -49, -49, -49, -50, -50, -49, -50, -50, -50, -51, -51, -51, -52, -53, -53, -52, -52, -53, -53, -53, -54, -54, -55, -55, -56, -56, -56, -56, -57, -56, -58, -58, -58, -59, -59, -60, -61, -61, -62, -63, -63, -62, -63, -63, -64, -65, -64, -65, -64, -65, -65, -66, -66, -66, -66, -67, -68, -68, -69, -68, -70, -69, -70, -70, -70, -71, -71, -72, -72, -73, -73, -73, -74, -73, -76, -76, -76, -77, -76, -78, -78, -78, -79, -78, -78, -79, -79, -80, -81, -81, -81, -82, -83, -83, -84, -84, -85, -86, -86, -86, -87, -87, -87, -87, -88, -88, -90, -89, -89, -91, -91, -91, -92, -92, -94, -93, -94, -94, -95, -95, -96, -97, -97, -99, -98, -99, -99, -101, -101, -102, -102, -102, -103, -104, -104, -105, -105, -106, -107, -106, -107, -108, -108, -108, -109, -111, -110, -111, -112, -111, -517, -517, -518, -518, -519, -518, -519, -519, -519, -520, -520, -520, -521, -521, -521, -522, -522, -523, -523, -523, -524, -523, -524, -525, -525, -526, -525, -525, -526, -527, -527, -526, -527, -528, -528, -528, -528, -529, -529, -530, -529, -529, -531, -530, -531, -532, -531, -532, -532, -533, -532, -533, -532, -534, -535, -534, -534, -535, -536, -535, -537, -537, -537, -537, -538, -539, -540, -538, -539, -540, -540, -541, -541, -541, -542, -541, -543, -544, -544, -543, -544, -543, -544, -545, -545, -546, -545, -546, -547, -547, -548, -550, -549, -549, -548, -549, -550, -550, -549, -551, -551, -550, -552, -552, -552, -553, -553, -553, -553, -554, -555, -555, -556, -555, -556, -556, -558, -557, -557, -558, -558, -559, -560, -560, -560, -560, -561, -561, -560, -563, -561, -562, -562, -563, -563, -563, -565, -564, -564, -565, -565, -565, -566, -566, -568, -566, -567, -568, -569, -569, -569, -569, -570, -570, -571, -571, -571, -571, -571, -571, -571, -573, -573, -572, -573, -573, -574, -574, -574, -574, -573, -574, -574, -575, -574, -575, -575, -575, -576, -577, -577, -577, -578, -578, -578, -578, -579, -580, -579, -579, -581, -580, -581, -581, -582, -581, -582, -582, -583, -583, -584, -584, -584, -583, -584, -584, -585, -586, -586, -586, -586, -587, -586, -586, -586, -588, -589, -589, -589, -590, -590, -590, -777, -777, -776, -776, -776, -777, -777, -777, -778, -778, -778, -777, -777, -777, -777, -778, -778, -777, -778, -778, -778, -778, -778, -779, -778, -779, -779, -778, -779, -779, -779, -779, -779, -779, -779, -779, -779, -780, -780, -780, -781, -780, -780, -540, -539, -539, -538, -539, -538, -538, -538, -538, -538, -538, -538, -536, -536, -537, -536, -536, -536, -535, -535, -534, -534, -534, -534, -534, -533, -533, -533, -533, -532, -532, -532, -531, -532, -531, -532, -531, -530, -531, -531, -531, -530, -531, -531, -531, -531, -531, -531, -530, -531, -530, -529, -528, -529, -529, -529, -529, -528, -528, -528, -528, -528, -527, -783, -783, -782, -783, -781, -781, -781, -783, -782, -783, -782, -782, -782, -783, -782, -782, -781, -782, -782, -783, -781, -782, -782, -782, -783, -782, -783, -782, -783, -782, -792, -804, -805, -796, -784, -774, -776, -780, -780, -778, -780, -786, -794, -802, -803, -798, -788, -782, -778, -779, -780, -785, -790, -792, -794, -794, -793, -790, -787, -782, -779, -780, -783, -787, -789, -791, -792, -792, -791, -787, -784, -782, -782, -784, -786, -788, -790, -792, -791, -789, -788, -784, -784, -784, -785, -786, -787, -790, -791, -791, -791, -787, -786, -784, -784, -784, -785, -787, -789, -790, -791, -790, -788, -788, -785, -785, -786, -786, -787, -789, -789, -790, -789, -787, -786, -786, -786, -689, -687, -688, -686, -686, -686, -686, -686, -685, -685, -685, -685, -685, -683, -684, -683, -684, -684, -683, -682, -683, -684, -683, -683, -682, -683, -683, -682, -682, -682, -682, -682, -682, -681, -682, -681, -681, -681, -680, -680, -679, -679, -679, -679, -678, -680, -678, -678, -679, -678, -678, -677, -677, -677, -676, -677, -677, -676, -677, -676, -676, -676, -676, -675, -676, -676, -675, -675, -676, -676, -676, -675, -675, -676, -674, -674, -674, -673, -673, -674, -673, -672, -672, -672, -671, -671, -671, -670, -670, -670, -669, -668, -669, -668, -668, -668, -669, -667, -667, -667, -668, -667, -666, -667, -666, -667, -665, -665, -665, -666, -666, -665, -665, -663, -664, -664, -664, -663, -664, -664, -663, -663, -663, -662, -663, -663, -663, -662, -662, -662, -662, -662, -662, -661, -661, -661, -661, -660, -659, -661, -660, -660, -661, -660, -659, -658, -657, -657, -658, -657, -657, -658, -657, -656, -657, -656, -655, -654, -656, -655, -654, -655, -655, -655, -653, -653, -655, -653, -653, -653, -653, -653, -652, -652, -652, -652, -652, -651, -652, -651, -651, -651, -652, -652, -651, -651, -651, -650, -650, -650, -649, -650, -649, -650, -650, -650, -650, -650, -648, -648, -648, -649, -649, -648, -648, -647, -648, -648, -647, -647, -648, -647, -647, -647, -646, -647, -647, -646, -646, -646, -646, -459, -459, -458, -458, -458, -458, -458, -457, -458, -456, -457, -457, -456, -457, -456, -456, -455, -456, -455, -455, -455, -455, -455, -455, -454, -454, -453, -454, -453, -454, -452, -453, -453, -452, -451, -450, -451, -450, -449, -450, -449, -449, -449, -449, -448, -449, -449, -450, -449, -448, -448, -448, -448, -448, -448, -448, -447, -448, -447, -447, -446, -446, -447, -446, -446, -446, -446, -446, -446, -446, -445, -445, -445, -445, -445, -445, -445, -445, -445, -444, -444, -444, -444, -444, -444, -445, -443, -444, -444, -444, -443, -444, -445, -444, -443, -443, -444, -444, -443, -443, -442, -442, -442, -441, -442, -442, -443, -442, -443, -442, -442, -442, -441, -441, -442, -441, -440, -441, -440, -440, -440, -441, -440, -441, -441, -440, -439, -440, -440, -439, -440, -438, -439, -438, -438, -439, -439, -439, -438, -438, -437, -438, -439, -438, -437, -438, -438, -437, -437, -436, -437, -436, -436, -436, -434, -434, -435, -434, -435, -435, -434, -434, -433, -433, -432, -432, -432, -432, -432, -430, -430, -430, -429, -429, -428, -428, -429, -427, -428, -426, -426, -426, -425, -426, -425, -425, -426, -424, -425, -425, -424, -425, -425, -425, -425, -424, -424, -424, -423, -422, -422, -422, -422, -422, -422, -422, -421, -421, -421, -420, -420, -420, -420, -420, -419, -418, -418, -418, -419, -417, -418, -417, -417, -325, -325, -324, -325, -325, -324, -324, -324, -323, -323, -323, -323, -323, -322, -322, -323, -322, -322, -322, -321, -322, -321, -321, -322, -322, -323, -323, -323, -324, -323, -324, -323, -324, -324, -324, -325, -324, -325, -324, -324, -324, -324, -324, -324, -323, -324, -324, -324, -324, -324, -323, -323, -323, -322, -323, -323, -323, -323, -324, -323, -322, -323, -322, -323, -322, -322, -322, -323, -322, -321, -322, -322, -322, -322, -322, -322, -322, -322, -322, -321, -322, -321, -322, -321, -322, -322, -321, -321, -320, -321, -320, -320, -320, -321, -321, -321, -321, -321, -322, -321, -322, -322, -322, -323, -323, -322, -321, -322, -322, -323, -322, -322, -322, -321, -322, -323, -323, -322, -322, -322, -323, -323, -322, -322, -323, -323, -323, -322, -322, -321, -323, -322, -322, -321, -322, -322, -321, -322, -322, -322, -322, -322, -322, -322, -322, -322, -323, -323, -322, -322, -322, -322, -322, -322, -322, -321, -322, -322, -323, -322, -322, -322, -322, -322, -322, -323, -322, -322, -322, -322, -321, -321, -320, -320, -319, -321, -321, -321, -320, -320, -320, -320, -320, -321, -321, -321, -320, -320, -319, -319, -319, -321, -320, -321, -321, -321, -321, -320, -320, -322, -322, -323, -323, -324, -323, -323, -324, -324, -324, -324, -325, -324, -324, -324, -324, -324, -324, -323, -323, -323, -324, -257, -267, -271, -258, -235, -219, -214, -223, -231, -229, -214, -195, -183, -181, -179, -171, -156, -142, -136, -138, -139, -131, -115, -97, -88, -89, -95, -95, -89, -83, -85, -91, -97, -99, -97, -99, -112, -135, -159, -175, -181, -185, -196, -218, -245, -271, -290, -308, -327, -351, -379, -406, -427, -449, -480, -514, -551, -583, -604, -620, -640, -671, -707, -743, -770, -784, -791, -785, -766, -721, -650, -583, -558, -585, -644, -675, -627, -505, -371, -294, -314, -393, -443, -409, -301, -193, -158, -205, -273, -278, -190, -69, -7, -57, -178, -268, -248, -135, -12, 27, -35, -136, -188, -159, -94, -61, -94, -158, -190, -158, -89, -46, -70, -144, -210, -223, -181, -128, -108, -133, -177, -205, -200, -184, -184, -216, -262, -286, -269, -225, -189, -194, -241, -296, -320, -303, -271, -257, -271, -296, -306, -293, -275, -277, -303, -330, -339, -315, -279, -259, -266, -285, -292, -279, -257, -243, -244, -244, -230, -195, -156, -138, -144, -158, -157, -132, -97, -71, -64, -67, -65, -48, -27, -19, -28, -41, -40, -21, 2, 562, 598, 670, 701, 602, 433, 257, 121, -11, -180, -306, -337, -269, -175, -136, -135, -160, -200, -254, -415, -624, -833, -1002, -1111, -1232, -1333, -1345, -1290, -1185, -1143, -1180, -1265, -1401, -1493, -1499, -970, 167, 1707, 2914, 3070, 2482, 1847, 1740, 1901, 1569, 503, -787, -1499, 152, 210, 146, 41, -1, 20, 21, -89, -244, -302, -209, -68, -14, -87, -222, 548, 518, 366, 192, 85, 76, 140, 221, 297, 333, 333, 300, 259, 168, 49, -113, -228, -275, -228, -183, -169, -255, -409, -547, -677, -771, -853, -984, -1140, -1224, -1276, -1218, -1250, -1348, -1473, -1528, -1586, -1575, -1660, -1709, -1659, -1062, 472, 2444, 3876, 3366, 1324, -688, -1052, 300, 1774, 1737, 298, -997, -775, 503, 1197, 320, -1452, -2580, -2147, -809, 421, 927, 899, 941, 1193, 1358, 1236, 824, 472, 471, 751, 1115, 1384, 1391, 1185, 755, 123, -613, -1225, -1394, -987, -329, 90, -8, -331, -341, 100, 538, 464, -113, -501, -96, 950, 1854, 1965, 1351, 675, 428, 495, 441, 80, -346, -478, -265, 24, 121, 6, -228, -476, -675, -740, -564, -126, 401, 811, 1011, 1107, 1162, 1155, 958, 593, 313, 265, 572, 845, 983, 849, 581, 384, 197, 62, -2, 43, 229, 502, 783, 1123, 1383, 1474, 1370, 1061, 848, 811, 943, 1110, 1118, 1026, 977, 934, 851, 629, 315, 100, 44, 135, 324, 482, 572, 585, 518, 437, 338, 257, 207, 169, 180, 269, 389, 461, 389, 176, -67, -248, -356, -413, -434, -450, -386, -355, -379, -417, -578, -733, -979, -1223, -1376, -1428, -1438, -1373, -1364, -1349, -1318, -1422, -1510, -1697, -1873, -1456, -124, 1888, 3763, 3848, 2279, 296, -575, 243, 1377, 1318, 95, -1137, -949, 446, 1405, 929, -669, -2099, -2188, -1313, -300, 352, 575, 885, 1431, 1841, 1907, 1513, 905, 513, 429, 573, 837, 1027, 1137, 1118, 835, 229, -575, -1204, -1296, -940, -545, -456, -520, -275, 415, 1094, 1141, 470, -268, -350, 330, 1165, 1504, 1299, 991, 961, 1114, 1002, 459, -248, 772, 830, 828, 727, 590, 462, 390, 344, 270, 176, 98, 75, 105, 123, 98, 24, -73, -152, -218, -259, -285, -312, -370, -420, -444, -443, -444, -496, -618, -760, -922, -1061, -1164, -1249, -1295, -1348, -1371, -1398, -1417, -1380, -1023, -162, 1081, 2215, 2650, 2419, 2043, 1963, 2106, 1929, 1138, 3, -864, -1101, -890, -660, -610, -572, -358, -16, 222, 245, 160, 122, 183, 295, 413, 502, 635, 917, 1357, 1799, 1997, 1840, 1475, 1121, 862, 575, 102, -513, -1000, -1138, -959, -715, -587, -480, -220, 185, 519, 607, 512, 441, 501, 624, 684, 666, 683, 815, 1022, 1176, 1177, 1031, 806, 551, 263, -71, -421, -719, -886, -896, -786, -619, -414, -164, 150, 459, 717, 830, 870, 1001, 1176, 1319, 1251, 1035, 879, 860, 838, 764, 663, 523, 443, 303, 213, 212, 224, 276, 317, 369, 499, 704, 974, 1201, 1295, 1325, 1394, 1448, 1393, 1236, 1070, 934, 803, 648, 526, 495, 475, 447, 429, 420, 452, 495, 512, 507, 466, 436, 484, 537, 557, 555, 552, 567, 574, 535, 495, 454, 377, 265, 152, 61, 23, -10, -63, -103, -113, -83, -26, -4, -19, -48, -93, -149, -224, -334, -451, -563, -689, -816, -926, -1010, -1044, -1048, -1067, -1083, -1112, -1171, -1230, -1296, -1324, -1114, -489, 517, 1590, 2224, 2308, 2171, 2167, 2313, 2224, 1598, 596, -346, -884, -1019, -1019, -1044, -1003, -771, -381, -9, 208, 320, 426, 527, 581, 567, 494, 454, 560, 846, 1230, 1513, 1557, 1447, 1332, 1251, 1097, 709, 107, -493, -884, -1026, -1049, -1075, -1036, -795, -356, 110, 435, 610, 749, 904, 1007, 983, 838, 684, 627, 664, 722, 745, 720, 684, 650, 574, 414, 160, -151, -434, -642, -1119, -843, -599, -383, -184, -7, 122, 166, 114, 3, -122, -237, -349, -473, -603, -706, -771, -811, -844, -864, -863, -847, -849, -893, -956, -1003, -1018, -1021, -1038, -1055, -1040, -971, -855, -712, -551, -380, -210, -56, 67, 160, 212, 219, 153, 51, -108, -288, -479, -671, -836, -988, -1102, -1161, -1170, -1118, -1021, -887, -726, -564, -412, -260, -129, -28, 20, 33, 11, -44, -125, -221, -318, -417, -519, -599, -657, -692, -718, -728, -723, -705, -681, -647, -604, -574, -555, -538, -511, -478, -460, -451, -448, -456, -468, -481, -485, -493, -516, -534, -559, -578, -599, -619, -633, -663, -701, -742, -780, -806, -829, -852, -865, -873, -875, -863, -838, -801, -761, -716, -676, -638, -606, -578, -549, -538, -546, -566, -597, -638, -682, -726, -762, -791, -818, -835, -840, -834, -819, -795, -766, -740, -718, -701, -688, -682, -686, -694, -714, -744, -779, -818, -851, -887, -921, -949, -977, -999, -1018, -1024, -1029, -1038, -1049, -1067, -1085, -1107, -1133, -1150, -1171, -1199, -1242, -1286, -1292, -1214, -1039, -816, -620, -469, -320, -132, 61, 198, 248, 229, 166, 51, -132, -367, -616, -832, -1001, -1147, -1292, -1413, -1450, -1376, -1218, -1031, -845, -661, -486, -316, -152, -9, 89, 114, 74, -2, -96, -193, -286, -370, -455, -544, -619, -650, -630, -584, -528, -482, -445, -417, -391, -357, -327, -316, -333, -361, -381, -384, -365, -321, -269, 1137, 1141, 1147, 1151, 1154, 1155, 1154, 1153, 1152, 1149, 1148, 1148, 1146, 1143, 1140, 1141, 1142, 1145, 1146, 1147, 1149, 1150, 1151, 1153, 1153, 1153, 1153, 1151, 1150, 1147, 1147, 1146, 1144, 1141, 1138, 1138, 1138, 1136, 1136, 1136, 1139, 1140, 1140, 1140, 1138, 1137, 1133, 1130, 1125, 1117, 1110, 1103, 1095, 1087, 1083, 1075, 1070, 1066, 1064, 1064, 1065, 1068, 1072, 1075, 1078, 1083, 1087, 1089, 1090, 1090, 1089, 1086, 1082, 1078, 1076, 1071, 1068, 1065, 1063, 1062, 1061, 1063, 1068, 1072, 1076, 1079, 1083, 1088, 1091, 1093, 1097, 1099, 1100, 1098, 1099, 1099, 1098, 1097, 1095, 1091, 1090, 1089, 1090, 1091, 1091, 1095, 1100, 1104, 1110, 1115, 1123, 1128, 1132, 1134, 1136, 1136, 1137, 1134, 1131, 1126, 1123, 1119, 1117, 1113, 1112, 1109, 1110, 1113, 1116, 1123, 1132, 1141, 1148, 1153, 1162, 1171, 1175, 1179, 1181, 1183, 1182, 1183, 1182, 1181, 1179, 1178, 1178, 1178, 1177, 1179, 1180, 1181, 1183, 1186, 1188, 1192, 1193, 1194, 1197, 1197, 1196, 1193, 1192, 1192, 1187, 1181, 1175, 1170, 1163, 1159, 1155, 1150, 1141, 1132, 1127, 1123, 1120, 1117, 1118, 1115, 1114, 1119, 1127, 1139, 1151, 1163, 1178, 1193, 1208, 1227, 1247, 1264, 1281, 1298, 933, 932, 931, 930, 928, 926, 925, 924, 923, 920, 918, 917, 915, 912, 910, 908, 905, 903, 901, 899, 896, 894, 891, 889, 888, 885, 883, 882, 880, 878, 877, 875, 873, 872, 870, 869, 868, 867, 867, 866, 865, 865, 864, 863, 863, 862, 863, 862, 862, 862, 863, 863, 863, 861, 862, 863, 862, 862, 863, 863, 862, 1465, 1468, 1469, 1468, 1465, 1463, 1461, 1458, 1456, 1456, 1455, 1456, 1458, 1460, 1463, 1468, 1474, 1478, 1482, 1486, 1489, 1490, 1493, 1497, 1497, 1496, 1494, 1495, 1496, 1496, 1496, 1498, 1500, 1503, 1507, 1509, 1513, 1517, 1520, 1525, 1527, 1530, 1533, 1533, 1535, 1534, 1536, 1539, 1540, 1542, 1542, 1546, 1552, 1556, 1560, 1563, 1569, 1572, 1576, 1580, 1584, 1586, 1588, 1590, 1592, 1593, 1593, 1593, 1594, 1591, 1591, 1590, 1590, 1588, 1593, 1596, 1601, 1605, 1611, 1615, 1618, 1622, 1624, 1626, 1627, 1627, 1627, 1626, 1623, 1620, 1618, 1616, 1615, 1614, 1614, 1614, 1616, 1617, 1619, 1621, 1622, 1623, 1625, 1624, 1626, 1627, 1627, 1629, 1629, 1631, 1631, 1634, 1635, 1636, 1636, 1637, 1636, 1634, 1633, 1630, 1628, 1626, 1623, 1619, 1618, 1615, 1614, 1615, 1616, 1616, 1617, 1616, 1617, 1615, 1614, 1612, 1609, 1606, 1601, 1597, 1592, 1589, 1581, 1575, 1568, 1561, 1556, 1551, 1547, 1545, 1543, 1543, 1543, 1543, 1546, 1547, 1548, 1549, 1550, 1549, 1549, 1547, 1545, 1543, 1541, 1537, 1535, 1532, 1528, 1525, 1524, 1521, 1521, 1522, 1524, 1525, 1528, 1532, 1536, 1539, 1543, 1547, 1550, 1552, 1556, 1558, 1559, 1559, 1558, 1558, 1556, 1555, 1554, 1554, 1553, 1553, 1553, 1554, 1554, 1555, 1557, 1557, 1558, 1561, 1562, 1565, 1570, 1574, 1576, 1581, 1583, 1587, 1590, 1591, 1593, 1594, 1597, 1599, 1601, 1602, 1606, 1608, 1610, 1610, 1612, 1611, 1612, 1611, 1609, 1609, 1608, 1607, 1604, 1602, 1599, 1599, 1598, 1599, 1598, 1598, 1598, 1599, 1600, 1600, 1601, 1601, 1600, 1599, 1598, 1595, 1593, 1590, 1589, 1587, 1585, 1585, 1584, 1583, 1582, 1581, 1580, 1579, 1578, 1576, 1576, 1576, 1576, 1577, 1579, 1581, 1580, 1582, 1582, 1583, 1583, 1585, 1587, 1589, 1590, 1592, 1594, 1595, 1596, 1594, 1592, 1590, 1587, 1586, 1584, 1584, 1582, 1582, 1580, 1578, 1576, 1574, 1570, 1568, 1565, 1561, 1558, 1555, 1554, 1550, 1546, 1544, 1539, 1536, 1533, 1528, 1525, 1523, 1521, 1518, 1515, 1514, 1515, 1514, 1514, 1516, 1518, 1521, 1523, 1526, 1528, 1531, 1533, 1538, 1540, 1543, 1546, 1548, 1550, 1553, 1554, 1557, 1559, 1561, 1563, 1566, 1568, 1571, 1574, 1578, 1580, 1583, 1587, 1590, 1592, 1596, 1598, 1600, 1601, 1602, 1603, 1603, 1603, 1602, 1601, 1597, 1595, 1591, 1588, 1585, 1581, 1577, 1573, 1569, 1566, 1560, 1555, 1550, 1544, 1539, 1535, 1531, 1525, 1523, 1520, 1518, 1517, 1516, 1514, 1512, 1512, 1511, 1512, 1512, 1513, 1512, 1515, 1516, 1516, 1517, 1516, 1513, 1509, 1505, 1501, 1496, 1489, 1480, 1473, 1463, 812, 812, 812, 812, 812, 812, 811, 812, 812, 812, 813, 813, 814, 814, 814, 814, 814, 815, 814, 814, 815, 815, 814, 815, 814, 815, 814, 814, 815, 814, 815, 815, 816, 816, 815, 816, 816, 816, 817, 816, 817, 816, 817, 818, 817, 817, 816, 817, 816, 817, 817, 818, 817, 817, 818, 818, 818, 818, 819, 818, 820, 820, 820, 819, 819, 820, 820, 819, 819, 820, 819, 820, 820, 819, 819, 819, 819, 819, 820, 819, 819, 819, 819, 820, 820, 820, 820, 820, 820, 821, 820, 821, 821, 820, 820, 821, 821, 821, 821, 820, 821, 822, 821, 821, 822, 822, 822, 822, 823, 824, 822, 822, 821, 822, 822, 823, 823, 823, 822, 824, 824, 823, 823, 824, 824, 824, 825, 825, 826, 825, 826, 826, 825, 827, 827, 827, 826, 827, 827, 827, 828, 828, 829, 829, 829, 829, 829, 831, 831, 830, 830, 832, 832, 831, 831, 832, 831, 832, 831, 832, 833, 832, 833, 832, 834, 833, 834, 834, 834, 834, 835, 835, 836, 836, 835, 835, 836, 835, 837, 836, 836, 836, 835, 835, 835, 835, 836, 836, 836, 836, 837, 837, 836, 837, 838, 837, 837, 838, 839, 838, 839, 837, 838, 838, 840, 839, 839, 839, 839, 840, 839, 840, 840, 841, 841, 842, 840, 841, 841, 841, 841, 843, 842, 842, 842, 842, 843, 842, 843, 842, 842, 842, 844, 844, 844, 844, 844, 843, 844, 844, 844, 844, 844, 844, 845, 844, 845, 844, 846, 844, 845, 845, 846, 846, 846, 847, 848, 848, 848, 848, 849, 848, 850, 850, 850, 851, 851, 850, 851, 850, 851, 851, 851, 851, 852, 852, 852, 851, 853, 852, 852, 853, 853, 854, 853, 855, 855, 855, 856, 855, 856, 856, 856, 856, 857, 858, 857, 858, 858, 858, 859, 859, 859, 859, 860, 859, 859, 860, 860, 860, 860, 860, 861, 862, 862, 861, 862, 862, 862, 862, 863, 863, 863, 863, 862, 863, 864, 862, 863, 862, 140, 140, 139, 138, 137, 136, 135, 135, 133, 132, 133, 130, 129, 129, 127, 127, 126, 125, 123, 123, 121, 120, 120, 119, 118, 118, 116, 116, 115, 113, 113, 112, 111, 109, 108, 108, 108, 108, 106, 104, 103, 103, 103, 102, 101, 99, 98, 97, 96, 96, 96, 96, 94, 94, 92, 92, 92, 90, 90, 88, 88, 86, 85, 85, 83, 82, 81, 80, 80, 79, 77, 77, 75, 75, 74, 74, 73, 72, 71, 71, 69, 68, 68, 67, 67, 67, 65, 65, 65, 62, 62, 61, 60, 59, 58, 58, 57, 55, 55, 53, 53, 52, 51, 50, 49, 49, 47, 47, 46, 45, 44, 43, 42, 42, 41, 40, 39, 39, 37, 36, 36, 35, 34, 33, 31, 30, 30, 29, 29, 27, 26, 26, 26, 24, 23, 22, 22, 22, 20, 19, 19, 18, 18, 18, 16, 16, 14, 14, 13, 12, 11, 11, 10, 9, 7, 6, 6, 5, 5, 4, 3, 2, 2, 1, 0, -2, -2, -3, -4, -5, -6, -7, -7, -8, -9, -9, -11, -11, -13, -13, -14, -14, -15, -16, -18, -19, -19, -20, -21, -21, -22, -24, -24, -25, -26, -27, -27, -28, -29, -29, -30, -31, -32, -32, -34, -35, -36, -36, -38, -38, -39, -40, -42, -42, -42, -43, -43, -45, -45, -47, -47, -49, -50, -50, -52, -52, -53, -54, -55, -749, -750, -749, -749, -750, -750, -749, -750, -749, -749, -749, -748, -749, -749, -749, -749, -749, -748, -749, -749, -748, -749, -748, -748, -748, -748, -748, -748, -749, -748, -749, -749, -749, -749, -750, -750, -749, -750, -750, -750, -751, -750, -750, -750, -750, -750, -750, -751, -752, -752, -752, -752, -753, -752, -753, -753, -753, -753, -753, -753, -753, -753, -752, -596, -597, -597, -597, -598, -599, -598, -599, -599, -599, -600, -601, -602, -602, -601, -602, -602, -603, -604, -604, -604, -604, -604, -605, -605, -605, -606, -607, -607, -607, -607, -607, -608, -608, -609, -609, -610, -609, -610, -610, -610, -611, -611, -612, -612, -613, -613, -612, -613, -613, -613, -613, -614, -614, -614, -615, -615, -615, -615, -617, -617, -617, -616, -617, -617, -618, -619, -618, -619, -620, -619, -619, -619, -620, -620, -621, -622, -622, -622, -622, -622, -621, -622, -622, -621, -622, -622, -623, -624, -625, -624, -624, -624, -624, -625, -625, -625, -625, -626, -626, -626, -626, -626, -626, -626, -627, -627, -628, -629, -628, -628, -628, -629, -629, -629, -628, -628, -628, -629, -631, -630, -631, -630, -630, -631, -631, -632, -631, -633, -632, -633, -633, -633, -633, -633, -634, -635, -635, -634, -636, -635, -636, -636, -636, -636, -637, -637, -637, -637, -637, -637, -637, -637, -638, -637, -637, -638, -638, -639, -639, -640, -639, -640, -642, -641, -641, -642, -641, -642, -641, -642, -642, -641, -642, -642, -643, -644, -643, -643, -644, -644, -644, -645, -645, -645, -646, -645, -645, -645, -645, -646, -647, -647, -647, -648, -647, -649, -648, -648, -648, -649, -650, -650, -651, -650, -755, -755, -755, -754, -755, -755, -754, -753, -754, -754, -754, -753, -754, -754, -754, -754, -753, -754, -754, -754, -754, -754, -754, -755, -754, -754, -754, -754, -753, -754, -753, -752, -754, -753, -753, -754, -754, -753, -753, -753, -753, -753, -753, -753, -752, -752, -753, -753, -752, -753, -753, -754, -753, -753, -753, -753, -753, -752, -753, -752, -753, -752, -753, -753, -753, -753, -753, -752, -752, -752, -752, -752, -752, -752, -752, -752, -752, -751, -751, -752, -751, -752, -752, -752, -752, -751, -752, -753, -753, -753, -753, -753, -754, -753, -753, -754, -753, -753, -753, -753, -753, -753, -752, -752, -753, -752, -752, -752, -753, -751, -752, -752, -752, -752, -752, -752, -752, -751, -751, -751, -751, -751, -750, -750, -750, -750, -751, -751, -750, -751, -750, -751, -750, -750, -750, -751, -751, -751, -750, -750, -750, -750, -749, -750, -750, -749, -750, -750, -751, -750, -750, -749, -750, -750, -749, -750, -749, -749, -749, -750, -749, -750, -749, -749, -750, -750, -750, -749, -749, -748, -750, -750, -750, -750, -749, -750, -749, -749, -749, -748, -748, -749, -749, -749, -748, -748, -748, -748, -749, -748, -748, -748, -748, -749, -748, -749, -749, -748, -748, -748, -747, -748, -747, -748, -748, -747, -747, -747, -747, -748, -747, -746, -747, -747, -747, -748, -747, -747, -747, -748, -783, -785, -785, -784, -785, -785, -785, -785, -785, -785, -785, -785, -785, -784, -785, -785, -785, -785, -786, -785, -785, -785, -785, -785, -785, -786, -785, -787, -786, -786, -786, -786, -786, -786, -786, -786, -787, -786, -785, -786, -786, -786, -787, -786, -787, -787, -788, -787, -787, -788, -788, -788, -788, -788, -788, -788, -787, -788, -787, -788, -789, -789, -788, -788, -788, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -789, -788, -789, -789, -788, -789, -789, -789, -789, -790, -789, -788, -790, -789, -789, -789, -790, -789, -789, -789, -790, -790, -789, -790, -790, -791, -791, -791, -791, -792, -791, -791, -790, -791, -790, -791, -791, -791, -791, -791, -792, -792, -791, -791, -791, -791, -791, -791, -791, -791, -792, -791, -790, -791, -791, -791, -792, -791, -791, -791, -791, -791, -791, -791, -791, -792, -792, -791, -792, -791, -792, -791, -791, -792, -791, -792, -792, -793, -792, -792, -793, -793, -793, -792, -792, -793, -793, -792, -793, -794, -793, -793, -792, -793, -794, -793, -794, -793, -793, -793, -792, -791, -793, -793, -793, -793, -791, -792, -792, -792, -793, -792, -792, -792, -791, -793, -792, -792, -791, -792, -792, -791, -791, -792, -792, -791, -791, -791, -792, -791, -792, -792, -792, -792, -792, -791, -792, -793, -792, -793, -792, -658, -659, -658, -657, -657, -656, -656, -655, -655, -655, -654, -654, -653, -654, -653, -652, -652, -651, -652, -651, -652, -652, -650, -651, -650, -649, -649, -649, -648, -648, -647, -648, -647, -646, -646, -646, -646, -645, -645, -645, -645, -645, -645, -643, -644, -643, -642, -642, -642, -642, -642, -642, -642, -643, -642, -642, -642, -642, -641, -641, -641, -640, -640, -640, -640, -639, -639, -638, -638, -639, -639, -639, -639, -637, -637, -637, -637, -636, -636, -636, -636, -636, -635, -635, -635, -635, -635, -634, -634, -633, -634, -632, -632, -632, -631, -630, -631, -630, -630, -630, -629, -629, -629, -628, -628, -627, -627, -626, -626, -626, -625, -625, -624, -625, -624, -625, -625, -625, -624, -623, -623, -622, -623, -622, -621, -621, -621, -620, -620, -619, -620, -619, -618, -619, -619, -618, -619, -618, -617, -618, -617, -617, -617, -616, -616, -615, -616, -616, -614, -614, -614, -613, -613, -614, -614, -613, -612, -613, -612, -611, -612, -611, -610, -611, -610, -609, -609, -608, -609, -608, -608, -608, -607, -607, -606, -606, -606, -605, -605, -605, -604, -603, -604, -604, -604, -604, -604, -604, -603, -603, -603, -602, -603, -603, -601, -602, -602, -601, -601, -602, -601, -600, -601, -601, -600, -600, -599, -599, -598, -597, -597, -597, -597, -596, -595, -595, -595, -594, -594, -594, -403, -404, -404, -404, -404, -404, -404, -404, -404, -404, -404, -405, -404, -404, -404, -403, -403, -404, -403, -403, -403, -404, -404, -403, -404, -403, -404, -403, -403, -404, -404, -403, -404, -404, -404, -404, -405, -405, -405, -405, -405, -404, -404, -404, -404, -404, -405, -404, -405, -404, -405, -405, -405, -404, -405, -406, -406, -405, -405, -405, -404, -404, -405, -405, -405, -405, -405, -405, -404, -405, -406, -404, -405, -405, -405, -404, -405, -405, -404, -404, -404, -404, -404, -403, -404, -404, -404, -404, -403, -404, -404, -404, -404, -404, -403, -74, -88, -90, -85, -83, -92, -113, -143, -173, -199, -216, -225, -231, -240, -255, -277, -306, -338, -369, -394, -414, -433, -452, -473, -493, -513, -540, -572, -604, -631, -655, -670, -681, -697, -715, -740, -762, -774, -772, -744, -682, -596, -514, -479, -502, -560, -592, -550, -432, -296, -217, -222, -290, -343, -326, -247, -154, -111, -136, -183, -190, -399, -400, -400, -401, -400, -401, -400, -400, -400, -401, -400, -400, -400, -400, -401, -401, -401, -401, -401, -401, -401, -401, -401, -401, -400, -400, -401, -400, -401, -402, -401, -401, -401, -402, -402, -401, -401, -401, -401, -401, -402, -403, -402, -403, -403, -403, -402, -403, -403, -403, -403, -402, -402, -402, -403, -404, -403, -403, -402, -403, -403, -402, -402, -402, -402, -370, -369, -368, -368, -368, -369, -369, -371, -371, -370, -369, -368, -369, -369, -367, -370, -371, -372, -371, -371, -370, -369, -369, -369, -371, -373, -373, -371, -371, -369, -369, -369, -369, -370, -368, -369, -368, -368, -368, -368, -368, -367, -367, -368, -368, -369, -368, -368, -368, -368, -369, -367, -367, -368, -369, -369, -368, -367, -368, -368, -366, -368, -368, -367, -367, -367, -367, -368, -367, -368, -367, -366, -367, -366, -366, -366, -367, -367, -368, -366, -367, -367, -369, -368, -369, -369, -369, -369, -370, -371, -372, -372, -372, -373, -373, -373, -372, -372, -373, -373, -372, -372, -372, -372, -373, -373, -371, -370, -371, -371, -371, -372, -372, -373, -371, -372, -372, -371, -370, -369, -369, -370, -370, -369, -368, -367, -367, -368, -367, -368, -367, -367, -365, -366, -367, -367, -366, -366, -365, -365, -365, -364, -364, -364, -364, -364, -365, -365, -364, -363, -362, -363, -364, -364, -363, -364, -364, -364, -365, -365, -366, -366, -366, -366, -366, -367, -366, -366, -367, -367, -367, -368, -367, -367, -367, -367, -368, -368, -367, -366, -368, -367, -367, -368, -368, -367, -368, -368, -370, -368, -367, -366, -365, -364, -362, -364, -363, -363, -363, -363, -362, -359, -357, -357, -359, -359, -360, -359, -570, -604, -627, -641, -651, -660, -682, -710, -741, -769, -788, -800, -799, -771, -702, -588, -465, -403, -442, -547, -626, -592, -433, -227, -89, -85, -186, -283, -290, -202, -83, -12, -27, -84, -108, -46, 75, 163, 140, 9, -140, -191, -93, 91, 232, 231, 103, -41, -96, -46, 34, 56, 5, -57, -52, 29, 112, 111, 6, -132, -209, -178, -76, 15, 32, -23, -99, -146, -155, -157, -176, -212, -235, -214, -150, -85, -72, -128, -221, -291, -299, -245, -173, -130, -130, -152, -172, -180, -187, -203, -226, -232, -204, -146, -86, -61, -79, -121, -157, -165, -138, -94, -49, -17, 7, 21, 25, 13, -6, -14, -3, 31, 75, 111, 126, 120, 104, 90, 79, 72, 74, 87, 107, 130, 137, 115, 71, 19, -14, -23, -13, -3, -4, -23, -51, -83, -113, -143, -171, -196, -210, -214, -215, -222, -245, -281, -327, -367, -396, -409, -416, -426, -440, -459, -493, -535, -572, -603, -629, -644, -654, -663, -681, -703, -723, -745, -766, -783, -777, -738, -653, -522, -397, -356, -420, -529, -585, -519, -340, -144, -34, -57, -157, -227, -206, -114, -15, 26, 0, -48, -49, 26, 145, 215, 179, 47, -94, -142, -49, 126, 262, 264, 144, 8, -48, -12, 54, 73, 26, -30, -25, 49, 128, 132, 42, -86, -173, -169, -90, 4, 46, 12, -62, -125, -152, -148, -151, -169, -198, -198, -152, -81, -41, -75, -169, -258, -285, -236, -147, -78, -67, 1129, 2717, 3335, 2410, 651, -678, -489, 721, 1490, 1026, -193, -801, -153, 792, 838, -290, -1690, -2057, -1219, 68, 988, 1142, 982, 989, 1158, 1299, 1133, 682, 348, 289, 598, 1083, 1241, 1086, 565, 11, -439, -809, -1012, -939, -587, -120, 145, 108, 31, 168, 451, 547, 255, -130, -61, 588, 1396, 1732, 1358, 713, 319, 291, 341, 143, -237, -472, -381, -58, 235, 300, 181, -3, -92, -50, 110, 373, 595, 777, 827, 919, 1062, 1136, 1038, 718, 406, 306, 418, 617, 700, 620, 512, 465, 493, 546, 545, 556, 610, 695, 849, 1014, 1151, 1192, 1093, 933, 824, 797, 828, 831, 747, 687, 665, 700, 700, 590, 433, 302, 259, 298, 358, 396, 417, 406, 373, 335, 293, 264, 211, 137, 90, 94, 139, 168, 112, 9, -90, -143, -167, -213, -290, -382, -411, -405, -362, -403, -453, -557, -644, -769, -908, -962, -1046, -1070, -1145, -1116, -1047, -1022, -1051, -1200, -1345, -1456, -1434, -1196, -434, 942, 2571, 3438, 2772, 1141, -204, -276, 540, 1024, 537, -452, -784, -43, 945, 1046, 34, -1245, -1756, -1270, -347, 404, 717, 867, 1163, 1571, 1829, 1632, 1027, 372, 39, 171, 572, 857, 853, 655, 409, 133, -274, -745, -1029, -960, -634, -331, -176, 16, 433, 926, 1080, 696, 108, -117, 220, 815, 1154, 1059, 789, 673, 755, 739, 396, -176, -662, -799, -600, -250, 42, 186, 212, 216, 257, 403, 540, 624, 560, 488, 613, 925, 1248, 1285, 1011, 639, 393, 352, 368, 312, 202, 177, 309, 542, 734, 831, 830, 757, 705, 736, 873, 1038, 1115, 1093, 1036, 1012, 1036, 1007, 843, 603, 411, 384, 469, 558, 593, 567, 538, 522, 507, 486, 437, 381, 339, 318, 350, 413, 450, 417, 314, 212, 156, 116, 71, 1, -65, -67, -20, 32, 24, -1165, -1179, -1163, -917, -250, 778, 1743, 2162, 2005, 1704, 1620, 1711, 1521, 829, -145, -888, -1118, -989, -863, -851, -780, -482, -53, 267, 363, 334, 305, 347, 415, 431, 372, 326, 435, 729, 1062, 1241, 1187, 998, 807, 665, 459, 67, -462, -898, -1054, -980, -885, -847, -729, -397, 75, 466, 621, 605, 594, 666, 743, 695, 518, 360, 334, 432, 533, 559, 500, 435, 358, 258, 84, -150, -373, -527, -613, -569, -408, -125, 184, 432, 638, 831, 988, 1088, 1073, 973, 820, 647, 498, 378, 296, 286, 333, 391, 418, 435, 495, 573, 595, 549, 483, 449, 442, 452, 486, 545, 630, 721, 812, 885, 918, 910, 863, 763, 631, 505, 411, 331, 239, 168, 158, 206, 273, 317, 341, 353, 346, 320, 272, 215, 163, 125, 101, 86, 85, 111, 167, 225, 263, 273, 260, 225, 167, 88, -3, -93, -181, -253, -303, -324, -307, -281, -251, -235, -228, -236, -265, -344, 280, 278, 278, 278, 276, 274, 271, 268, 265, 265, 264, 264, 264, 263, 262, 262, 263, 265, 265, 266, 266, 265, 262, 260, 259, 260, 260, 262, 263, 266, 269, 274, 279, 280, 282, 283, 281, 277, 273, 268, 263, 256, 247, 237, 231, 225, 220, 218, 215, 213, 213, 215, 218, 223, 226, 229, 231, 230, 231, 230, 229, 225, 218, -1258, -1011, -556, -126, 147, 348, 599, 888, 1085, 1074, 879, 652, 502, 427, 355, 258, 172, 145, 150, 193, 203, 175, 67, -86, -236, -346, -417, -366, -222, -20, 146, 281, 490, 768, 1009, 1134, 1124, 1029, 914, 775, 651, 502, 340, 219, 160, 158, 195, 281, 409, 509, 543, 580, 642, 697, 670, 595, 555, 562, 589, 626, 655, 690, 715, 746, 777, 759, 691, 600, 493, 374, 249, 162, 128, 99, 61, 53, 97, 176, 236, 268, 285, 289, 276, 251, 208, -937, -1032, -1107, -1158, -1184, -1193, -1185, -1154, -1098, -1028, -958, -903, -861, -829, -807, -801, -820, -850, -878, -906, -935, -969, -989, -997, -1002, -1001, -992, -987, -979, -983, -984, -996, -1013, -1031, -1049, -1075, -1102, -1116, -1115, -1108, -1091, -1065, -1029, -990, -958, -928, -901, -886, -889, -910, -944, -989, -1035, -1077, -1116, -1160, -1198, -1216, -1217, -1209, -1195, -1174, -1143, -1113, -1089, -1070, -1057, -1052, -1054, -1072, -1100, -1131, -1160, -1183, -1202, -1219, -1229, -1233, -1230, -1222, -1212, -1211, -1213, -1216, -1228, -1260, -1295, -1332, -1368, -1420, -1471, -1514, -1548, -1588, -1611, -1617, -1600, -1581, -1548, -1463, -1293, -1054, -807, -619, -494, -395, -279, -175, -150, -226, -365, -517, -677, -875, -1098, -1288, -1408, -1464, -1505, -1545, -1544, -1466, -1311, -1125, -959, -824, -711, -612, -530, -475, -447, -444, -477, -545, -639, -721, -772, -802, -839, -892, -943, -972, -980, -979, -982, -987, -995, -1006, -1021, -1028, -1017, -985, -952, -917, -878, -817, -731, -634, -547, -481, -434, -398, -380, -392, -432, -495, -574, -667, -774, -876, -959, -1021, -1068, -1096, -1102, -1082, -1039, -985, -924, -863, -802, -742, -693, -661, -649, -643, -640, -642, -654, -671, -686, -700, -715, -728, -735, -740, -751, -774, -805, -838, -866, -890, -911, -933, -952, 408, 405, 402, 397, 390, 383, 378, 372, 364, 359, 354, 350, 346, 346, 345, 346, 346, 347, 345, 345, 347, 350, 355, 358, 362, 366, 368, 370, 373, 376, 378, 379, 380, 379, 378, 378, 379, 378, 377, 372, 367, 363, 361, 358, 355, 352, 351, 352, 350, 349, 348, 346, 344, 342, 338, 333, 331, 327, 326, 321, 317, 316, 313, 311, 308, 308, 307, 307, 309, 310, 311, 315, 317, 321, 322, 323, 324, 324, 326, 328, 328, 330, 330, 330, 331, 332, 335, 338, 341, 346, 349, 353, 359, 364, 371, 376, 380, 384, 386, 390, 394, 398, 401, 403, 407, 411, 416, 421, 426, 431, 436, 438, 443, 444, 446, 449, 451, 452, 453, 451, 450, 447, 444, 439, 435, 428, 423, 417, 411, 406, 400, 397, 393, 392, 389, 387, 385, 384, 384, 383, 382, 380, 375, 372, 368, 366, 360, 355, 345, 336, 326, 315, 307, 298, 286, 274, 263, 251, 240, 232, 232, 232, 237, 236, 234, 238, 232, 235, 233, 231, 235, 227, 223, 218, 209, 210, 214, 211, 214, 207, 208, 211, 212, 210, 204, 195, 179, 162, 143, 126, 106, 85, 70, 61, 57, 64, 82, 102, 128, 158, 191, 228, 262, 294, 326, 350, 371, 388, 398, 404, 406, 403, 402, 396, 388, 379, 369, 356, 337, 319, 295, 267, 235, 203, 171, 141, 112, 85, 61, 39, 22, 9, 3, 0, 3, 13, 29, 53, 81, 116, 153, 193, 238, 283, 329, 376, 420, 461, 500, 533, 563, 590, 611, 628, 642, 653, 662, 668, 671, 676, 675, 671, 663, 652, 633, 611, 586, 560, 534, 507, 483, 459, 438, 418, 405, 392, 381, 372, 364, 358, 354, 353, 352, 353, 349, 347, 343, 338, 334, 327, 320, 311, 299, 285, 270, 254, 235, 215, 193, 173, 155, 142, 131, 128, 126, 132, 141, 155, 170, 187, 206, 226, 248, 268, 288, 303, 315, 324, 333, 337, 340, 340, 338, 332, 326, 319, 652, 653, 654, 654, 655, 657, 658, 659, 659, 661, 662, 664, 664, 665, 666, 666, 666, 666, 667, 667, 668, 668, 667, 669, 669, 669, 668, 669, 666, 667, 668, 667, 668, 667, 664, 665, 664, 663, 662, 661, 661, 660, 658, 657, 655, 654, 653, 651, 650, 649, 647, 645, 644, 641, 641, 639, 638, 638, 638, 637, 635, 636, 635, 635, 634, 634, 634, 634, 634, 634, 635, 634, 634, 634, 634, 634, 635, 635, 636, 636, 638, 636, 637, 637, 637, 638, 639, 639, 640, 640, 641, 642, 642, 643, 643, 643, 642, 643, 643, 643, 642, 642, 642, 641, 641, 641, 640, 640, 640, 639, 639, 639, 640, 640, 639, 639, 639, 638, 638, 639, 639, 639, 639, 640, 638, 640, 639, 639, 640, 640, 640, 640, 640, 642, 642, 642, 643, 644, 644, 646, 646, 645, 647, 647, 647, 647, 648, 648, 647, 648, 649, 649, 650, 651, 650, 651, 651, 651, 653, 653, 654, 654, 655, 656, 656, 657, 658, 658, 659, 659, 660, 660, 661, 660, 661, 662, 663, 663, 664, 665, 665, 666, 667, 668, 668, 670, 670, 671, 671, 672, 673, 674, 674, 676, 676, 677, 678, 679, 679, 679, 680, 680, 682, 682, 684, 685, 685, 686, 687, 688, 688, 689, 690, 689, 690, 691, 692, 692, 694, 694, 695, 696, 697, 698, 698, 700, 699, 701, 701, 701, 702, 703, 704, 704, 704, 705, 706, 706, 707, 707, 708, 709, 709, 710, 710, 711, 711, 711, 712, 713, 713, 714, 715, 714, 717, 717, 718, 718, 719, 721, 720, 721, 721, 722, 723, 723, 724, 725, 726, 725, 725, 727, 727, 728, 728, 728, 729, 730, 730, 731, 732, 733, 733, 734, 735, 735, 736, 737, 738, 739, 739, 740, 740, 740, 741, 742, 743, 744, 744, 744, 745, 746, 746, 748, 748, 749, 750, 750, 750, 751, 753, 753, 755, 755, 756, 757, 758, 759, 759, 759, 760, 761, 761, 763, 763, 763, 764, 765, 765, 765, 767, 711, 711, 710, 710, 710, 710, 710, 710, 709, 709, 709, 709, 709, 708, 225, 225, 224, 223, 223, 223, 222, 223, 221, 221, 221, 219, 220, 217, 217, 217, 216, 217, 216, 215, 214, 214, 213, 213, 211, 211, 211, 210, 210, 210, 210, 209, 209, 208, 208, 207, 206, 205, 205, 205, 204, 204, 203, 203, 203, 202, 202, 202, 202, 200, 200, 200, 199, 199, 199, 198, 197, 197, 197, 197, 196, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 698, 699, 698, 699, 699, 699, 699, 700, 699, 699, 698, 699, 700, 699, 699, 699, 699, 699, 699, 700, 700, 699, 699, 699, 699, 700, 700, 700, 699, 700, 699, 700, 700, 700, 699, 699, 698, 698, 698, 698, 699, 699, 700, 699, 699, 700, 699, 700, 700, 700, 701, 700, 701, 702, 701, 702, 701, 702, 702, 702, 702, 702, 702, 702, 703, 703, 703, 703, 704, 704, 704, 704, 704, 704, 705, 705, 705, 705, 705, 705, 705, 706, 707, 708, 708, 708, 708, 709, 709, 709, 710, 709, 710, 710, 710, 711, 711, 712, 711, 711, 712, 712, 713, 713, 713, 713, 714, 714, 714, 713, 714, 714, 715, 715, 715, 715, 716, 714, 715, 716, 717, 717, 717, 717, 717, 718, 718, 718, 718, 718, 719, 720, 720, 721, 721, 720, 722, 722, 722, 722, 723, 724, 723, 723, 724, 723, 725, 724, 724, 726, 725, 726, 726, 726, 725, 726, 727, 728, 727, 728, 729, 728, 729, 729, 729, 730, 730, 731, 732, 732, 731, 731, 733, 733, 734, 734, 735, 735, 735, 735, 736, 736, 736, 736, 737, 736, 738, 739, 739, 739, 739, 740, 740, 741, 742, 742, 743, 743, 742, 743, 745, 745, 746, 746, 746, 748, 748, 749, 748, 750, 750, 751, 751, 750, 752, 752, 753, 754, 754, 754, 755, 755, 756, 756, 758, 757, 758, 758, 759, 759, 759, 760, 761, 762, 762, 763, 762, 764, 764, 764, 765, 765, 765, 614, 614, 613, 613, 612, 611, 610, 609, 610, 610, 609, 609, 609, 608, 607, 606, 606, 605, 604, 604, 604, 602, 602, 601, 600, 601, 600, 600, 599, 599, 598, 597, 597, 596, 597, 596, 595, 595, 595, 594, 593, 592, 592, 591, 592, 590, 590, 590, 589, 589, 588, 588, 587, 586, 587, 586, 586, 585, 585, 584, 583, 583, 582, 582, 581, 580, 581, 579, 580, 580, 578, 578, 578, 576, 576, 576, 575, 575, 574, 573, 572, 572, 571, 571, 571, 570, 570, 569, 568, 568, 568, 569, 567, 567, 566, 566, 566, 565, 565, 563, 562, 563, 562, 561, 561, 560, 559, 559, 558, 558, 557, 557, 557, 556, 556, 555, 554, 554, 552, 552, 552, 551, 549, 549, 549, 548, 547, 547, 546, 545, 545, 545, 544, 543, 542, 542, 541, 540, 539, 539, 539, 539, 537, 537, 535, 535, 535, 534, 533, 534, 532, 532, 531, 530, 530, 529, 529, 529, 527, 528, 527, 526, 526, 525, 524, 524, 524, 523, 524, 523, 522, 521, 521, 521, 521, 520, 518, 518, 518, 519, 517, 516, 516, 516, 515, 515, 514, 514, 512, 513, 513, 512, 512, 512, 511, 511, 511, 509, 509, 507, 508, 508, 508, 506, 506, 505, 505, 505, 504, 503, 502, 503, 503, 502, 501, 501, 500, 500, 499, 498, 498, 498, 498, 498, 498, 497, 496, 497, 496, 495, 495, 494, 493, 492, 492, 493, 490, 491, 490, 489, 489, 489, 487, 487, 487, 486, 485, 486, 485, 485, 485, 485, 484, 483, 482, 482, 481, 481, 480, 479, 479, 478, 477, 478, 476, 476, 475, 473, 473, 473, 472, 472, 471, 470, 470, 469, 469, 468, 467, 466, 465, 465, 464, 464, 464, 462, 462, 461, 460, 459, 459, 458, 456, 457, 455, 454, 455, 454, 453, 454, 452, 451, 452, 450, 450, 449, 450, 449, 448, 448, 446, 445, 445, 444, 444, 444, 444, 443, 442, 442, 441, 440, 439, 439, 439, 437, 436, 437, 436, 435, 433, -14, -14, -14, -13, -14, -14, -14, -15, -16, -14, -14, -15, -16, -16, -16, -16, -16, -16, -16, -16, -15, -15, -15, -16, -14, -15, -16, -17, -16, -17, -17, -18, -17, -18, -19, -18, -18, -19, -19, -19, -19, -19, -19, -19, -19, -19, -20, -20, -19, -20, -20, -20, -21, -21, -21, -21, -21, -22, -22, -23, -21, -22, -23, -23, -22, -22, -23, -24, -22, -23, -23, -23, -23, -22, -23, -22, -24, -24, -25, -24, -25, -25, -25, -25, -25, -25, -25, -25, -25, -26, -25, -25, -26, -26, -26, -26, -25, -26, -26, -26, -26, -26, -26, -26, -26, -26, -26, -26, -27, -27, -26, -27, -27, -27, -28, -28, -28, -29, -29, -28, -28, -28, -29, -29, -29, -30, -30, -31, -31, -30, -30, -30, -30, -31, -31, -31, -32, -32, -32, -32, -32, -32, -34, -34, -34, -34, -34, -34, -34, -34, -34, -35, -34, -35, -35, -35, -35, -35, -36, -36, -37, -37, -36, -37, -37, -37, -38, -38, -37, -38, -37, -38, -37, -37, -38, -39, -39, -39, -39, -39, -40, -40, -40, -40, -40, -40, -40, -41, -41, -41, -42, -43, -42, -43, -42, -42, -42, -43, -43, -44, -45, -45, -44, -45, -45, -44, -46, -46, -46, -47, -48, -49, -48, -49, -48, -47, -49, -49, -49, -49, -254, -255, -255, -254, -255, -257, -257, -256, -257, -258, -257, -258, -258, -259, -259, -258, -258, -259, -260, -260, -259, -261, -260, -260, -259, -260, -260, -260, -260, -260, -260, -261, -261, -261, -262, -261, -261, -262, -262, -262, -263, -263, -263, -263, -263, -262, -263, -263, -262, -262, -264, -263, -263, -263, -263, -263, -264, -264, -263, -265, -264, -265, -265, -264, -265, -264, -264, -264, -265, -265, -265, -266, -265, -265, -267, -265, -266, -267, -267, -267, -267, -267, -266, -267, -267, -266, -267, -267, -267, -267, -268, -268, -268, -268, -269, -268, -269, -268, -268, -268, -268, -269, -270, -269, -270, -270, -270, -270, -271, -270, -269, -270, -269, -270, -269, -271, -270, -270, -271, -270, -271, -271, -272, -272, -272, -272, -272, -273, -272, -273, -272, -273, -274, -275, -274, -273, -274, -274, -274, -274, -274, -274, -274, -274, -276, -275, -276, -276, -276, -276, -277, -278, -278, -279, -278, -278, -623, -623, -623, -624, -623, -623, -621, -621, -622, -621, -621, -622, -622, -622, -622, -622, -620, -621, -622, -622, -621, -622, -622, -622, -622, -622, -622, -622, -622, -622, -623, -623, -624, -623, -624, -623, -624, -623, -623, -624, -624, -624, -624, -625, -625, -626, -626, -625, -625, -625, -627, -626, -627, -626, -627, -627, -628, -628, -627, -626, -627, -626, -627, -291, -291, -507, -506, -507, -507, -507, -507, -508, -509, -509, -510, -509, -510, -510, -511, -511, -510, -512, -511, -512, -512, -513, -513, -513, -514, -514, -514, -514, -514, -516, -515, -516, -516, -516, -517, -518, -518, -518, -519, -519, -519, -520, -520, -521, -520, -521, -519, -521, -521, -521, -522, -523, -522, -523, -523, -523, -523, -524, -524, -524, -524, -524, -525, -525, -526, -525, -525, -527, -526, -526, -526, -527, -526, -527, -527, -526, -527, -527, -527, -527, -527, -527, -528, -528, -528, -528, -528, -528, -529, -529, -530, -529, -530, -531, -530, -530, -531, -531, -531, -532, -532, -532, -532, -533, -533, -534, -533, -533, -534, -534, -534, -534, -534, -534, -534, -534, -534, -535, -534, -535, -535, -535, -536, -536, -536, -535, -536, -536, -536, -536, -536, -537, -536, -537, -536, -537, -537, -537, -536, -537, -536, -537, -536, -537, -537, -537, -538, -538, -539, -538, -537, -537, -538, -538, -539, -538, -539, -539, -539, -539, -540, -540, -540, -540, -541, -540, -540, -541, -541, -542, -542, -542, -542, -543, -543, -542, -542, -543, -543, -544, -543, -544, -544, -543, -543, -543, -543, -543, -544, -544, -544, -544, -544, -544, -545, -546, -545, -546, -546, -545, -545, -545, -545, -545, -546, -545, -546, -546, -546, -545, -547, -545, -546, -547, -547, -547, -547, -547, -547, -548, -547, -622, -622, -622, -621, -622, -623, -621, -621, -621, -620, -620, -620, -619, -619, -619, -619, -619, -618, -619, -618, -619, -619, -618, -618, -618, -618, -618, -618, -618, -618, -619, -618, -618, -617, -617, -617, -616, -617, -616, -617, -617, -617, -617, -617, -616, -617, -618, -618, -618, -617, -617, -617, -618, -618, -618, -618, -619, -618, -619, -619, -619, -619, -620, -619, -620, -621, -620, -621, -621, -621, -622, -622, -621, -622, -622, -623, -623, -623, -622, -621, -622, -621, -622, -622, -622, -622, -621, -622, -622, -620, -620, -620, -621, -620, -622, -621, -621, -620, -620, -619, -619, -619, -619, -621, -620, -621, -620, -619, -619, -620, -620, -619, -620, -620, -620, -620, -620, -620, -619, -620, -618, -619, -619, -619, -618, -618, -618, -618, -618, -618, -619, -619, -619, -618, -618, -618, -618, -619, -618, -617, -617, -619, -617, -618, -617, -616, -618, -617, -617, -617, -617, -617, -618, -617, -617, -616, -616, -616, -616, -616, -615, -615, -615, -615, -615, -615, -615, -614, -616, -615, -615, -615, -615, -616, -615, -615, -614, -615, -616, -615, -614, -615, -615, -614, -614, -614, -615, -614, -614, -614, -613, -614, -614, -613, -612, -613, -614, -613, -614, -613, -613, -614, -613, -614, -613, -614, -614, -614, -615, -615, -614, -614, -613, -615, -615, -613, -614, -614, -614, -614, -614, -482, -483, -483, -483, -483, -484, -484, -482, -483, -484, -484, -482, -483, -482, -482, -482, -481, -482, -482, -482, -482, -482, -482, -480, -480, -480, -480, -480, -481, -480, -480, -479, -479, -479, -479, -478, -479, -479, -478, -478, -478, -478, -478, -478, -476, -476, -476, -474, -475, -475, -475, -475, -474, -475, -473, -473, -473, -473, -474, -474, -473, -471, -471, -471, -472, -473, -473, -474, -473, -471, -472, -472, -472, -471, -471, -471, -472, -471, -472, -472, -472, -472, -470, -470, -471, -470, -470, -472, -470, -470, -470, -471, -472, -471, -471, -471, -472, -472, -471, -472, -471, -472, -472, -472, -472, -471, -471, -471, -471, -472, -470, -470, -471, -471, -470, -470, -471, -470, -469, -470, -470, -470, -470, -470, -470, -469, -469, -467, -468, -468, -468, -469, -468, -467, -466, -466, -465, -466, -467, -468, -469, -468, -468, -468, -467, -467, -467, -469, -468, -469, -469, -468, -467, -467, -467, -467, -466, -466, -468, -467, -468, -467, -467, -466, -465, -466, -466, -468, -468, -469, -468, -468, -466, -465, -463, -464, -465, -465, -464, -463, -462, -461, -460, -459, -460, -462, -461, -460, -460, -458, -459, -458, -458, -460, -460, -459, -458, -458, -458, -456, -455, -456, -455, -455, -455, -454, -454, -455, -453, -454, -451, -451, -450, -452, -453, -453, -453, -452, -452, -451, -450, 3, -76, -147, -121, -22, 33, -42, -216, -356, -353, -218, -61, -5, -82, -223, -325, -331, -283, -263, -305, -370, -388, -324, -221, -167, -222, -368, -505, -537, -447, -311, -228, -246, -328, -401, -410, -372, -343, -362, -409, -425, -379, -292, -227, -229, -299, -382, -415, -378, -296, -221, -185, -180, -184, -183, -177, -175, -174, -165, -128, -71, -19, 4, -8, -33, -51, -46, -28, -12, -3, 5, 23, 42, 46, 20, -29, -75, -90, -70, -39, -23, -39, -79, -123, -155, -172, -186, -204, -222, -232, -232, -234, -252, -293, -345, -388, -411, -411, -403, -405, -425, -457, -493, -528, -562, -593, -622, -643, -657, -668, -684, -715, -759, -805, -844, -874, -895, -913, -933, -949, -946, -906, -797, -644, -519, -503, -618, -785, -871, -769, -501, -206, -52, -121, -333, -517, -537, -383, -159, -16, -29, -145, -229, -181, -21, 116, 104, -69, -282, -358, -211, 73, 296, 297, 90, -159, -275, -206, -54, 40, 15, -71, -102, -38, 51, 53, -72, -245, -344, -299, -152, -17, 12, -76, -205, -295, -312, -290, -280, -304, -335, -329, -267, -187, -160, -229, -369, -491, -508, -411, -266, -175, -189, -281, -367, -388, -349, -304, -306, -349, -374, -328, -225, -134, -130, -219, -336, -382, -323, -192, -78, -44, -76, -107, -102, -65, -42, -56, -83, -74, 1, 103, 163, 135, 42, -35, -40, 22, 88, -1248, -265, 478, 821, 1014, 1332, 1689, 1778, 1438, 805, 256, 94, 305, 666, 913, 907, 693, 341, -108, -597, -1014, -1180, -1008, -647, -330, -136, 73, 453, 869, 981, 645, 130, -83, 246, 870, 1304, 1284, 964, 692, 609, 526, 212, -312, -761, -890, -681, -351, -92, 7, 8, -48, -90, -142, -22, 209, 534, 792, 879, 954, 1068, 1259, 1426, 1340, 957, 511, 188, 154, 216, -566, -589, -618, -651, -671, -693, -725, -770, -813, -855, -889, -925, -953, -977, -989, -997, -993, -974, -941, -903, -866, -835, -806, -784, -769, -773, -793, -834, -878, -931, -976, -1018, -1057, -1083, -1094, -1093, -1079, -1075, -1061, -1047, -1038, -1042, -1055, -1075, -1106, -1139, -1171, -1186, -1194, -1192, -1181, -1164, -1139, -1111, -1078, -1046, -1029, -1031, -1045, -1066, -1095, -822, -950, -1102, -1239, -1325, -1352, -1317, -1290, -1340, -1460, -1591, -1636, -1591, -1302, -418, 1103, 2775, 3632, 3042, 1532, 255, -17, 488, 776, 270, -571, -838, -167, 757, 921, 116, -989, -1536, -1243, -561, -8, 302, 587, 1115, 1787, 2200, 2065, 1435, 687, 227, 140, 258, 376, 397, 385, 371, 267, -18, -455, -837, -965, -869, -727, -610, -375, 157, 858, 1314, 1236, 769, 374, 376, 676, 894, 807, 557, 455, 605, 783, 670, 209, -358, -734, -830, -731, -592, -433, -250, -14, 190, 336, 360, 376, 341, 502, 740, 1035, 1423, 1573, 1589, 1285, 825, 491, 229, 46, -105, -216, -144, 93, 358, 605, 705, 656, 686, 726, 845, 1024, 1177, 1379, 1522, 1535, 1494, 1383, 1169, 890, 547, 259, 154, 164, 253, 314, 302, 373, 443, 500, 540, 489, 470, 471, 524, 629, 698, 48, 117, 168, 200, 194, 137, 79, 14, -73, -182, -277, -341, -381, -422, -443, -455, -472, -529, -579, -619, -657, -716, -776, -840, -856, -859, -829, -819, -832, -886, -904, -826, -507, 134, 937, 1596, 1834, 1679, 1490, 1422, 1380, 1093, 448, -294, -810, -959, -890, -805, -754, -592, -275, 75, 297, 343, 327, 359, 436, 491, 467, 404, 388, 493, 693, 892, 981, 922, 768, 601, 428, 202, -115, -462, -714, -793, -765, -704, -620, -436, -113, 256, 511, 589, 568, 586, 661, 692, 602, 442, 325, 320, 386, 429, 418, 365, 293, 202, 69, -103, -271, -404, -497, -555, -574, -416, -99, 296, 600, 757, 884, 1047, 1173, 1224, 1122, 959, 775, 599, 450, 335, 228, 174, 153, 147, 168, 228, 334, 426, 435, 430, 482, 593, 678, 702, 705, 750, 823, 888, 905, 877, 805, 706, 586, 458, 327, 225, 135, 61, 5, 6, 67, 165, 250, 302, 347, 407, 464, 487, 448, 397, 356, 310, 254, 186, 131, 91, 45, -2, -32, -46, -54, -67, -86, -96, -90, -68, -35, -25, -30, -32, -22, -13, -35, -92, -153, -226, -297, -387, -477, -552, -611, -680, -742, -783, -778, -773, -790, -819, -823, -809, -798, -821, -866, -920, -906, -768, -363, 272, 963, 1461, 1613, 1543, 1486, 1488, 1411, 1017, 353, -313, -724, -888, -954, -1013, -978, -752, -387, -44, 172, 287, 403, 537, 636, 624, 511, 378, 320, 378, 510, 637, 699, 684, 639, 596, 519, 349, 66, -263, -529, -694, -807, -896, -908, -755, -441, -95, 166, 338, 504, 703, 866, 892, 767, 579, 429, 342, 276, 189, 99, 42, 23, 11, -21, -75, -137, -214, -312, -757, -730, -704, -675, -635, -597, -558, -518, -472, -434, -410, -392, -367, -352, -351, -358, -363, -369, -387, -406, -414, -416, -421, -423, -415, -396, -376, -352, -320, -289, -260, -234, -206, -183, -171, -165, -157, -156, -167, -183, -198, -212, -233, -254, -271, -284, -302, -320, -334, -343, -354, -365, -375, -381, -387, -392, -397, -398, -399, -401, -401, -400, -402, -404, -406, -408, -414, -424, -431, -437, -449, -466, -485, -500, -514, -527, -542, -556, -570, -581, -591, -596, -599, -599, -597, -592, -583, -571, -556, -537, -521, -507, -496, -483, -474, -474, -479, -490, -501, -515, -535, -551, -565, -575, -587, -597, -604, -603, -602, -596, -593, -586, -586, -585, -593, -607, -626, -642, -657, -678, -705, -723, -731, -735, -742, -749, -754, -766, -789, -829, -869, -873, -805, -651, -455, -280, -142, -10, 143, 300, 410, 452, 437, 379, 276, 121, -74, -275, -445, -569, -672, -776, -865, -891, -827, -692, -531, -378, -245, -134, -30, 74, 176, 238, 237, 182, 104, 28, -33, -84, -129, -177, -232, -275, -291, -279, -251, -226, -218, -232, -255, -268, -270, -274, -288, -308, -317, -305, -271, -211, -131, -44, 38, 112, 182, 256, 313, 343, 334, 294, 235, 162, 68, -31, -129, -213, -280, -332, -358, -353, -317, -257, -182, -99, -8, 79, 156, 218, 263, 291, 304, 291, 257, 215, 175, 141, 105, 71, 46, 33, 30, 31, 40, 56, 70, 83, 467, 474, 479, 481, 489, 493, 496, 505, 501, 498, 493, 490, 489, 491, 492, 495, 497, 501, 505, 514, 520, 525, 526, 526, 527, 529, 529, 530, 531, 531, 537, 543, 551, 561, 576, 595, 617, 642, 668, 696, 722, 751, 779, 805, 831, 851, 869, 885, 899, 913, 928, 940, 947, 957, 970, 982, 998, 1015, 1033, 1051, 1065, 1077, 1088, 1096, 1102, 1105, 1103, 1098, 1094, 1088, 1083, 1077, 1072, 1069, 1068, 1068, 1073, 1081, 1089, 1098, 1105, 1111, 1117, 1119, 1118, 1113, 1104, 1091, 1073, 1051, 1027, 1003, 979, 955, 933, 910, 893, 880, 870, 867, 864, 865, 868, 872, 876, 879, 882, 884, 882, 879, 872, 863, 854, 838, 822, 804, 784, 767, 749, 734, 723, 716, 713, 716, 721, 731, 746, 761, 778, 795, 812, 828, 844, 856, 864, 868, 868, 864, 855, 842, 827, 807, 784, 761, 734, 704, 678, 652, 629, 610, 595, 586, 582, 582, 589, 603, 620, 640, 663, 686, 711, 735, 760, 782, 804, 824, 840, 857, 875, 892, 908, 927, 946, 968, 989, 1014, 1038, 1062, 1084, 1105, 1125, 1145, 1166, 1182, 1198, 1212, 1224, 1236, 1246, 1252, 1256, 1256, 1254, 1251, 1247, 1242, 1235, 1223, 1212, 1199, 1187, 1177, 1167, 1154, 1142, 1128, 1115, 1101, 1087, 1072, 1056, 1042, 1028, 1015, 1001, 990, 982, 972, 964, 955, 947, 937, 928, 922, 916, 910, 904, 896, 890, 885, 880, 873, 868, 861, 854, 849, 842, 837, 829, 824, 819, 815, 813, 811, 812, 812, 809, 804, 797, 789, 782, 773, 767, 760, 749, 736, 719, 697, 672, 643, 610, 577, 546, 519, 499, 487, 483, 484, 490, 498, 506, 513, 516, 515, 510, 504, 498, 496, 499, 515, 542, 582, 629, 685, 744, 804, 861, 915, 965, 1009, 1051, 1087, 1118, 1144, 1166, 1183, 1198, 1209, 1218, 1222, 1223, 1225, 1226, 1227, 1227, 1228, 1227, 1227, 1228, 1229, 1230, 1229, 1225, 1218, 1209, 1194, 1173, 943, 941, 943, 943, 944, 946, 946, 947, 947, 948, 947, 948, 950, 950, 801, 801, 800, 799, 799, 799, 800, 799, 798, 798, 798, 797, 796, 796, 794, 795, 794, 794, 792, 793, 792, 792, 791, 791, 790, 790, 790, 791, 790, 790, 789, 789, 788, 788, 787, 787, 787, 786, 786, 786, 784, 785, 784, 785, 785, 784, 784, 644, 645, 644, 641, 636, 632, 624, 617, 607, 598, 587, 577, 567, 556, 546, 538, 531, 525, 521, 518, 517, 518, 521, 524, 530, 536, 543, 550, 557, 565, 572, 578, 584, 591, 596, 601, 607, 612, 616, 620, 623, 627, 630, 633, 635, 637, 639, 642, 643, 645, 646, 647, 649, 651, 651, 654, 656, 657, 659, 660, 662, 663, 664, 666, 667, 668, 668, 670, 671, 671, 671, 672, 673, 672, 673, 672, 672, 672, 672, 672, 671, 670, 670, 669, 669, 668, 667, 666, 665, 665, 664, 664, 664, 663, 662, 662, 662, 662, 662, 663, 662, 663, 662, 663, 664, 663, 664, 664, 664, 664, 665, 664, 664, 665, 664, 664, 664, 664, 663, 663, 662, 662, 662, 660, 660, 660, 658, 656, 656, 654, 653, 652, 651, 650, 647, 646, 645, 643, 641, 639, 638, 637, 635, 633, 630, 628, 626, 624, 622, 618, 616, 614, 611, 609, 608, 604, 602, 600, 596, 594, 591, 588, 586, 583, 580, 578, 574, 571, 569, 566, 563, 560, 558, 554, 552, 549, 546, 543, 540, 538, 534, 532, 529, 526, 525, 522, 519, 517, 515, 513, 512, 510, 508, 507, 506, 505, 504, 503, 503, 501, 500, 500, 499, 499, 499, 498, 498, 497, 497, 496, 496, 495, 495, 495, 495, 495, 495, 494, 494, 494, 494, 493, 494, 494, 494, 494, 495, 494, 495, 493, 492, 493, 492, 491, 491, 491, 491, 490, 491, 490, 490, 489, 489, 489, 487, 487, 485, 486, 486, 485, 484, 484, 484, 483, 483, 482, 482, 482, 481, 481, 480, 480, 480, 479, 479, 478, 478, 478, 477, 478, 477, 478, 477, 478, 477, 477, 477, 478, 478, 477, 478, 477, 477, 478, 1030, 1029, 1029, 1028, 1028, 1028, 1027, 1028, 1026, 1027, 1027, 1027, 1027, 1026, 1027, 1026, 1026, 1025, 1025, 1026, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1023, 1023, 1023, 1023, 1022, 1022, 1021, 1022, 1022, 1020, 1020, 1020, 1020, 1021, 1020, 1020, 1020, 1020, 1019, 1018, 1019, 1018, 1017, 1017, 1017, 1016, 1016, 1015, 1016, 1016, 1015, 1015, 1014, 1014, 1015, 1014, 1013, 1014, 1014, 1013, 1014, 1013, 1013, 1013, 1012, 1012, 1012, 1012, 1012, 1013, 1012, 1012, 1012, 1012, 1011, 1011, 1011, 1011, 1011, 1010, 1011, 1010, 1011, 1010, 1009, 1010, 1009, 1009, 1008, 1007, 1008, 1008, 1007, 1008, 1006, 1006, 1006, 1006, 1005, 1006, 1006, 1006, 1006, 1005, 1004, 1004, 1004, 1004, 1004, 1002, 1003, 1002, 1003, 1003, 1002, 1002, 1002, 1002, 1001, 1000, 999, 999, 1000, 999, 999, 999, 997, 998, 998, 998, 998, 997, 997, 995, 995, 995, 995, 994, 993, 993, 992, 992, 992, 992, 992, 990, 991, 990, 989, 989, 989, 988, 988, 987, 988, 987, 986, 986, 985, 985, 984, 984, 983, 984, 984, 983, 982, 982, 981, 982, 982, 981, 980, 979, 980, 979, 979, 978, 977, 977, 976, 976, 975, 975, 974, 974, 973, 973, 972, 972, 972, 971, 970, 971, 970, 970, 968, 969, 968, 967, 967, 967, 966, 966, 965, 966, 965, 964, 963, 963, 963, 962, 962, 961, 962, 961, 961, 960, 958, 959, 959, 958, 957, 957, 958, 957, 956, 955, 956, 955, 954, 954, 953, 953, 953, 953, 952, 952, 951, 951, 951, 950, 950, 949, 950, 948, 949, 948, 948, 948, 948, 946, 945, 945, 945, 945, 945, 945, 944, 944, 944, 943, 943, 943, 943, 944, 942, 942, 942, 942, 942, 941, 940, 940, 940, 939, 939, 938, 938, 937, 938, 938, 936, 935, 935, 935, 935, 935, 934, 935, 934, 934, 934, 933, 934, 932, 933, 933, 933, 933, 932, 931, 931, 930, 931, 931, 931, 931, 930, 930, 929, 929, 929, 929, 929, 929, 928, 928, 927, 927, 927, 462, 462, 462, 461, 461, 458, 458, 457, 457, 456, 454, 455, 454, 453, 453, 453, 452, 452, 450, 450, 450, 449, 448, 447, 448, 447, 447, 445, 445, 444, 443, 443, 441, 441, 440, 440, 440, 439, 439, 438, 438, 436, 435, 435, 435, 434, 434, 433, 433, 432, 431, 432, 430, 430, 428, 428, 428, 427, 426, 424, 425, 423, 423, 422, 423, 422, 422, 420, 420, 419, 418, 416, 415, 416, 415, 415, 413, 414, 413, 412, 411, 409, 409, 409, 408, 407, 408, 406, 406, 405, 404, 404, 403, 402, 402, 401, 400, 400, 399, 398, 397, 397, 397, 395, 394, 393, 393, 392, 392, 391, 392, 390, 389, 390, 388, 387, 386, 386, 386, 385, 384, 384, 382, 383, 381, 381, 380, 379, 379, 379, 377, 377, 377, 376, 376, 373, 372, 371, 373, 372, 372, 371, 371, 369, 368, 369, 367, 367, 366, 366, 366, 367, 365, 365, 365, 362, 363, 362, 361, 361, 360, 360, 359, 359, 357, 357, 356, 356, 356, 355, 355, 354, 352, 352, 352, 350, 352, 349, 348, 348, 347, 348, 346, 346, 345, 345, 344, 344, 343, 343, 342, 341, 342, 341, 341, 340, 339, 338, 339, 339, 338, 337, 335, 336, 335, 334, 333, 333, 332, 331, 330, 330, 329, 328, 327, 327, 327, 325, 325, 325, 325, 323, 323, 322, 320, 321, 320, 319, 318, 318, 317, 317, 316, 314, 314, 313, 313, 311, 311, 310, 310, 309, 309, 307, 307, 307, 305, 305, 305, 303, 303, 302, 302, 302, 301, 300, 300, 299, 298, 298, 297, 296, 296, 296, 296, 294, 293, 293, 291, 292, 290, 290, 289, 289, 288, 288, 287, 288, 286, 285, 285, 284, 284, 283, 282, 281, 281, 279, 278, 278, 278, 276, 276, 275, 275, 275, 273, 272, 272, 271, 271, 270, 269, 268, 268, 267, 265, 265, 264, 264, 263, 263, 262, 262, 261, 261, 261, 260, 259, 258, 257, 258, 257, 256, 256, 256, 255, 254, 254, 253, -187, -187, -186, -187, -187, -187, -187, -188, -188, -189, -189, -190, -190, -191, -191, -191, -192, -191, -192, -192, -193, -787, -788, -789, -790, -790, -791, -791, -792, -792, -793, -794, -795, -794, -795, -796, -797, -797, -798, -799, -800, -800, -800, -801, -802, -803, -803, -804, -805, -805, -806, -807, -807, -807, -808, -809, -810, -810, -811, -812, -813, -812, -813, -814, -814, -815, -815, -815, -816, -816, -817, -818, -819, -820, -820, -822, -821, -822, -823, -822, -823, -824, -211, -211, -211, -211, -211, -212, -212, -212, -212, -212, -213, -213, -213, -214, -213, -212, -213, -214, -214, -215, -214, -213, -214, -215, -215, -215, -215, -215, -215, -215, -216, -216, -215, -215, -216, -215, -216, -215, -215, -216, -216, -216, -216, -218, -217, -218, -218, -218, -219, -218, -219, -219, -220, -218, -219, -220, -221, -221, -221, -221, -222, -222, -222, -222, -222, -222, -223, -224, -224, -224, -224, -225, -225, -225, -226, -226, -226, -225, -225, -227, -226, -227, -227, -227, -228, -228, -228, -228, -228, -229, -229, -230, -229, -230, -230, -230, -232, -233, -232, -233, -233, -234, -233, -235, -234, -234, -234, -235, -235, -235, -235, -235, -235, -235, -237, -236, -237, -238, -237, -238, -238, -238, -238, -238, -239, -239, -240, -240, -240, -240, -240, -241, -241, -241, -240, -242, -242, -242, -368, -367, -368, -369, -369, -368, -368, -368, -369, -370, -370, -370, -369, -369, -370, -369, -371, -371, -371, -371, -371, -372, -370, -371, -371, -372, -372, -371, -371, -371, -371, -372, -371, -372, -372, -372, -372, -372, -372, -373, -373, -372, -373, -372, -373, -372, -373, -373, -372, -373, -373, -374, -373, -373, -373, -373, -373, -373, -373, -374, -373, -375, -374, -374, -374, -375, -375, -374, -374, -375, -375, -374, -374, -374, -375, -375, -375, -375, -375, -375, -375, -376, -375, -376, -376, -376, -376, -376, -376, -377, -375, -375, -376, -375, -375, -376, -377, -377, -378, -377, -378, -377, -377, -377, -376, -378, -377, -376, -378, -377, -377, -376, -377, -376, -377, -377, -378, -376, -377, -378, -376, -377, -378, -378, -378, -378, -378, -378, -380, -380, -380, -379, -380, -380, -380, -380, -379, -380, -380, -381, -381, -381, -381, -381, -382, -382, -382, -383, -382, -383, -383, -383, -384, -383, -384, -385, -384, -385, -386, -385, -385, -386, -387, -386, -386, -387, -386, -388, -387, -388, -388, -388, -388, -389, -389, -389, -389, -389, -389, -389, -390, -390, -390, -389, -390, -390, -391, -391, -391, -391, -391, -391, -393, -392, -392, -392, -392, -393, -394, -394, -394, -395, -395, -395, -395, -396, -395, -396, -397, -396, -397, -397, -397, -398, -397, -398, -398, -399, -399, -399, -754, -754, -754, -753, -753, -752, -751, -750, -750, -750, -748, -747, -746, -746, -746, -746, -744, -743, -744, -743, -743, -742, -742, -741, -740, -739, -739, -739, -739, -739, -739, -738, -738, -737, -737, -736, -735, -735, -733, -734, -733, -734, -732, -731, -731, -731, -730, -730, -730, -728, -727, -727, -726, -726, -726, -726, -725, -724, -723, -723, -724, -723, -721, -721, -721, -720, -719, -719, -718, -719, -716, -717, -716, -716, -715, -714, -714, -713, -712, -712, -711, -710, -710, -710, -710, -708, -708, -707, -707, -705, -705, -705, -704, -703, -703, -702, -702, -702, -700, -701, -700, -700, -698, -698, -698, -697, -696, -696, -695, -694, -694, -693, -692, -692, -691, -690, -690, -689, -688, -688, -687, -686, -686, -685, -684, -684, -683, -683, -682, -681, -681, -681, -679, -679, -678, -678, -678, -677, -676, -676, -676, -674, -673, -672, -672, -672, -670, -670, -669, -668, -668, -667, -666, -666, -665, -664, -663, -663, -663, -662, -662, -660, -661, -660, -659, -658, -658, -657, -656, -656, -655, -654, -654, -653, -652, -653, -651, -651, -650, -650, -649, -649, -649, -648, -648, -647, -646, -646, -645, -645, -645, -644, -643, -642, -642, -641, -640, -641, -640, -639, -638, -639, -637, -638, -637, -637, -636, -634, -635, -634, -634, -634, -633, -633, -632, -631, -631, -629, -630, -628, -628, -438, -439, -439, -439, -438, -437, -436, -436, -437, -436, -438, -438, -438, -439, -437, -437, -437, -437, -437, -437, -438, -438, -439, -439, -438, -439, -438, -439, -437, -438, -437, -438, -439, -438, -438, -437, -438, -438, -438, -437, -438, -438, -438, -438, -437, -437, -435, -434, -436, -436, -435, -435, -434, -434, -434, -433, -433, -433, -432, -433, -432, -432, -433, -433, -434, -432, -433, -434, -433, -434, -433, -433, -433, -433, -433, -431, -432, -431, -430, -430, -431, -431, -431, -431, -432, -431, -430, -431, -430, -429, -429, -429, -429, -429, -428, -428, -427, -428, -427, -427, -427, -425, -426, -426, -426, -427, -427, -428, -427, -426, -426, -427, -427, -428, -427, -427, -427, -428, -427, -427, -428, -428, -427, -427, -426, -425, -425, -426, -425, -425, -425, -425, -425, -424, -425, -426, -426, -427, -426, -427, -427, -427, -427, -427, -427, -427, -427, -427, -427, -427, -426, -426, -427, -426, -427, -427, -427, -429, -429, -428, -429, -429, -430, -431, -431, -431, -432, -432, -431, -432, -432, -432, -432, -430, -430, -430, -431, -429, -430, -430, -429, -429, -429, -428, -428, -428, -428, -427, -427, -427, -427, -426, -428, -428, -427, -427, -426, -426, -427, -428, -427, -428, -427, -428, -426, -426, -427, -428, -427, -425, -426, -426, -427, -426, -425, -424, -423, -424, -423, -423, -423, -360, -424, -452, -421, -355, -303, -304, -354, -416, -446, -433, -399, -377, -379, -393, -398, -384, -361, -353, -373, -403, -417, -398, -355, -309, -288, -293, -313, -327, -321, -301, -276, -252, -233, -218, -208, -201, -202, -208, -210, -202, -186, -167, -154, -146, -143, -149, -159, -176, -192, -199, -193, -177, -167, -173, -196, -229, -257, -269, -267, -264, -266, -278, -295, -309, -325, -338, -357, -376, -393, -403, -403, -402, -408, -427, -456, -488, -511, -526, -535, -542, -553, -572, -597, -625, -652, -677, -695, -709, -723, -741, -762, -785, -807, -826, -844, -856, -840, -768, -647, -541, -520, -609, -743, -803, -716, -509, -304, -228, -317, -479, -572, -520, -359, -202, -152, -224, -332, -364, -275, -126, -36, -82, -238, -386, -402, -253, -43, 75, 13, -180, -354, -383, -267, -120, -64, -135, -250, -297, -236, -138, -111, -194, -324, -398, -353, -224, -117, -113, -213, -340, -409, -393, -329, -283, -291, -335, -367, -354, -312, -290, -325, -399]

In [ ]:
print('length of time series: {}'.format(len(time_series)))
time_series_to_wav(time_series)

**2.** Define function to predict a single sample

In [ ]:
def predict_single_sample(model, X, y):
    """Predict a single sample using the trained model
    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))

# pick a sample to predict from the test set
X_to_predict = X_test[100]
y_to_predict = y_test[100]

# predict sample
predict_single_sample(model, X_to_predict, y_to_predict)